# Preamble

In [2]:
%matplotlib qt
bbox_inches='tight'

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec

def set_plot_style():
    """
    Apply consistent styling across all plots.
    Recommended to call this at the start of each notebook.
    """
    plt.rcParams.update({
        "font.family": "Helvetica",      # Change to 'Times New Roman' or 'Arial' if needed
        "font.size": 11,
        "axes.labelsize": 11,
        "axes.titlesize": 11,
        "xtick.labelsize": 10,
        "ytick.labelsize": 10,
        "legend.fontsize": 8,
        "figure.dpi": 300,
        "savefig.dpi": 300,
        "axes.grid": False,
        "grid.alpha": 0.3,
        "lines.linewidth": 0.8,
        "lines.markersize":3,
        "axes.spines.top": True,
        "axes.spines.right": True,
        "figure.figsize": (6, 3),  # Default for 1x2 layout
        "legend.frameon": False,
        "pdf.fonttype": 42,  # Embed fonts in PDF
        "ps.fonttype": 42
    })

def fig_row(cols=2, figsize=None, sharex=False, sharey=False):
    """
    Create a row of subplots.

    Parameters:
    - cols: number of columns
    - figsize: optional tuple (width, height)
    - sharex, sharey: whether to share x/y axes

    Returns:
    - fig, axes: figure and list of axes
    """
    if not figsize:
        figsize = (4 * cols,4)
    fig, axes = plt.subplots(1, cols, figsize=figsize, sharex=sharex, sharey=sharey)
    if cols == 1:
        axes = [axes]
    fig.tight_layout()
    fig.subplots_adjust(bottom=0.15,top=0.85)
    return fig, axes

def custom_layout():
    """
    Custom layout with a big left subplot and two stacked subplots on the right.

    Returns:
    - fig, (ax_big, ax1, ax2): figure and axes
    """
    fig = plt.figure(figsize=(8, 4))
    gs = GridSpec(2, 2, width_ratios=[2.5, 1],height_ratios=[1,1])

    ax_big = fig.add_subplot(gs[:, 0])     # large left
    ax1 = fig.add_subplot(gs[0, 1])        # top right
    ax2 = fig.add_subplot(gs[1, 1])        # bottom right
    fig.tight_layout()
    fig.subplots_adjust(bottom=0.15,top=0.85)
    return fig, (ax_big, ax1, ax2)

palette = {
    'ERA5L': 'lightskyblue',   # blue
    'Model': 'pink'   # red
}


# Intro figure 2: Observed wind and temperature

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# choose a good representative station 
set_plot_style()
name="Drangdrung"
# load data

data=pd.read_csv('Universaldatasets_smallerdomain/'+name+'.csv')
# Preprocess data
data['datetime'] = pd.to_datetime(data['datetime'])
# data=data[data['datetime'].dt.month>=8]
# data=data[data['datetime'].dt.month<9]
data.set_index('datetime', inplace=True)
# data['temp'] = data['temp'] - data['temp'].mean()
# data['ws'] = data['ws'] - data['ws'].mean()

# Plot temperature and wind with twin axis
golden = (1 + 5 ** 0.5) / 2


size = 10
# Create a new subplot for hourly grouped and averaged data
fig, ax = fig_row(cols=2,figsize=(10,5))
# plt.subplots_adjust(wspace=0.1,hspace=0.1,left=0.1,right=0.9,top=0.9,bottom=0.1)
# make the first subplot width double wide and the second subplot width normal


data['temp_model'][data['ws'].isna()]=np.nan
data['ws_model'][data['ws'].isna()]=np.nan
# Plot original data on the left subplot
l2 = ax[0].plot(data.index, data['temp_model'], linewidth=0.5, color='red',label=r"$T^{era}\ (^{\circ}\text{C})$",marker='x',markersize=0.5)
l1 = ax[0].plot(data.index, data['ws'], linewidth=0.5, color='purple', label=r"$u^{obs}\ (\text{ms}^{-1})$"  ,marker='x',markersize=0.5)
l3 = ax[0].plot(data.index, data['ws_model'], linewidth=0.5, color='dodgerblue', label=r"$u^{era}\ (\text{ms}^{-1})$"  ,marker='x',markersize=0.5,zorder=100)
ax[0].legend(frameon=False, shadow=True,fontsize=10,loc='upper left', bbox_to_anchor=(0.0, 0.95))
ax[0].set_xlabel("Date")
ax[0].set_xticks(pd.date_range(start="2023-05-30", end="2023-09-2", freq="1m"))
ax[0].tick_params(axis='both', which='major',rotation=0)
ax[0].tick_params(axis='y', which='major',rotation=90)
ax[1].tick_params(axis='y', which='major',rotation=90)

ax[0].set_ylim(-12,15)
# Group data by hour and average
hourly_data = data.groupby(data.index.hour).mean()
hourly_errors=data.groupby(data.index.hour).std()/np.sqrt(data.groupby(data.index.hour).count())
ax[0].text(0.05, 0.95, 'a.', transform=ax[0].transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
ax[1].text(0.05, 0.95, 'b.', transform=ax[1].transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')


# Plot hourly grouped and averaged data on the right subplot
l1 = ax[1].plot(hourly_data.index, hourly_data['temp_model'], linewidth=1, color='red', label=r"$T^{era}\ (^{\circ}\text{C})$",marker='o')
ax[1].fill_between(x=hourly_data.index,y1=hourly_data['temp_model']-hourly_errors['temp_model'],y2=hourly_data['temp_model']+hourly_errors['temp_model'],color='red',alpha=0.2,linewidth=0)
l2 = ax[1].plot(hourly_data.index, hourly_data['ws'], linewidth=1, color='purple', label=r"$u^{obs}\ (\text{ms}^{-1})$",marker='o')
ax[1].fill_between(x=hourly_data.index,y1=hourly_data['ws']-hourly_errors['ws'],y2=hourly_data['ws']+hourly_errors['ws'],color='purple',alpha=0.2,linewidth=0)
l3 = ax[1].plot(hourly_data.index, hourly_data['ws_model'], linewidth=1, color='dodgerblue', label=r"$u^{era}\ (\text{ms}^{-1})$",marker='o',zorder=100)
ax[1].fill_between(x=hourly_data.index,y1=hourly_data['ws_model']-hourly_errors['ws_model'],y2=hourly_data['ws_model']+hourly_errors['ws_model'],color='dodgerblue',alpha=0.2,linewidth=0)


ax[1].legend(frameon=False, shadow=True,fontsize=10,loc='upper left', bbox_to_anchor=(0.0,0.95))
ax[1].set_xticks(np.arange(0, 24, 5))
ax[1].tick_params(axis='both', which='major')
#set tick size to 3
ax[1].set_ylim(-2.9,6.9)
ax[1].set_xlabel("Local time (hr)")
plt.show()
# Save plot to PDF
plt.savefig("observed_wind_temperature.pdf",)
# plt.close() 
# 555

C:\Users\krish\AppData\Local\Temp\ipykernel_14908\2158923964.py:31: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['temp_model'][data['ws'].isna()]=np.nan
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\2158923964.py:31: SettingWithCop

## Model fit works in one station

In [4]:
set_plot_style()

from matplotlib.backends.backend_pdf import PdfPages
# import scienceplots
from sklearn.metrics import root_mean_squared_error,r2_score
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
# Load parameters and preprocess
params = pd.read_csv(r'New_bigtable/Universalparams.csv')
    
# Station names
# names = [
#      'schiaparelli',
# ]

# Open PDF file
pdf_path = "model_fit_works_one_stations.pdf"
params.set_index('glacier_name',inplace=True)
RMSE_ours=[]
RMSE_era=[]
params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)
with PdfPages(pdf_path) as pdf:
    for i, name in enumerate(tqdm(['Place3'])):
        leave = []
        
        
        
        # Obtaining predicted parameters
        
        S_pred=params_stats['S_fit'].loc[name]
        D_pred=params_stats['D_fit'].loc[name]
        U_pred=params['mean_wind_obs'].loc[name]
        
        se_S_pred=params_stats['se_S_fit'].loc[name]
        se_D_pred=params_stats['se_D_fit'].loc[name]
        se_U_pred=0
        
        A = S_pred
        B = -D_pred * S_pred

        
        # Getting timeseries data
        data_1=pd.read_csv('Universalhourlydatasets/'+name+'.csv')
        params=pd.read_csv('New_bigtable/Universalparams.csv')
        params.set_index('glacier_name',inplace=True)
        lat=params['lat'].loc[name]
        lon=params['lon'].loc[name]
        data_1=data_1.groupby(data_1.index%24).mean()
        linear_model=data_1[['temp_model','ws','temp','ws_model']]
        linear_model.rename(columns={'temp':'temp1'},inplace=True)
        linear_model.rename(columns={'temp_model':'temp'},inplace=True)
        mean_temp_cache=linear_model['temp'].mean()
        linear_model['temp']=linear_model['temp']-linear_model['temp'].mean()
        linear_model['temp1']=linear_model['temp1']-linear_model['temp1'].mean()
        mean_wind_cache=linear_model['ws'].mean()
        linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
        linear_model['ws_model1']=linear_model['ws_model']
        linear_model['dt']=linear_model['temp'].diff()
        linear_model=linear_model.dropna()

        # use the parameters to predict the wind speed
        linear_model['ws_pred']=A*linear_model['temp']+B*linear_model['dt']+U_pred
        linear_model['d_ws_pred']=np.sqrt(((linear_model['temp']-D_pred*linear_model['dt'])**2)*se_S_pred**2+((S_pred*linear_model['dt'])**2)*se_D_pred**2+se_U_pred**2) # u fit error is zero
        linear_model['ws_pred_upper']=linear_model['ws_pred']+linear_model['d_ws_pred']
        linear_model['ws_pred_lower']=linear_model['ws_pred']-linear_model['d_ws_pred']
        
        linear_model['ws']=linear_model['ws']+mean_wind_cache
        linear_model['temp']=linear_model['temp']+mean_temp_cache
        # Plot
        plt.ioff()
        fig,[ax,ax1]=fig_row(2,sharey=False,sharex=False)
        ax.text(0.05, 0.95, 'a.', transform=ax.transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
        ax1.text(0.05, 0.95, 'b.', transform=ax1.transAxes,
                fontsize=15, fontweight='bold', va='center', ha='center')

        l1=ax.plot(linear_model['ws'].rolling(1).mean().index,linear_model['ws'].rolling(1).mean(),linewidth=1,color='purple',marker='o',label=r"$u^{obs}$")
        l2=ax.plot(linear_model['ws_pred'].rolling(1).mean().index,linear_model['ws_pred'].rolling(1).mean(),linewidth=1,color='deeppink',marker='^',linestyle='--',label=r"$u^{pred}$")
        l3=ax.plot(linear_model['ws_model1'].rolling(1).mean().index,linear_model['ws_model1'].rolling(1).mean(),linewidth=1,color='darkorange',marker='^',linestyle='--',label=r"$u^{era}$")
        ax.fill_between(linear_model['ws_pred'].rolling(1).mean().index,linear_model['ws_pred_upper'].rolling(1).mean(),linear_model['ws_pred_lower'].rolling(1).mean(),alpha=0.5,color='deeppink',linewidth=0.1)
        ax.annotate("RMSE = "+str(round(root_mean_squared_error(linear_model['ws_pred'],linear_model['ws']),2))+r"$\ \text{ms}{}^{-1}$",(0,3.5),size=10)

        ax.set_ylabel("Wind speed (m/s)")
        
        lns=l1+l2+l3
        labs = [l.get_label() for l in lns]
        ax.legend(lns, labs)
        ax.set_xticks(ticks=np.arange(0,25,4))

        # ax.set_title(f"$\\bar{{u}}={U_pred:.2f}$, $s={S_pred:.2f} \\pm {se_S_pred:.2f}$, $\\tau={D_pred:.2f} \\pm {se_D_pred:.2f}$")
        ax.set_yticks(ticks=np.arange(0,4,0.5))
        
        ax.set_yticklabels(labels=np.round(np.arange(0,4,0.5),1),rotation=90)
        ax.set_ylim(0.2,4)
        ax.set_xlabel("Hour of day in UTC")
        plt.axes(ax1)
        # plt.rcParams["font.family"] = "Helvetica"
        # plt.rcParams['font.size']=20
        # plt.grid(True,zorder=-1)
        # plt.axhline(0,color='black',linewidth=1,linestyle='--')
        plt.axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
        
        plt.scatter(linear_model['ws'],linear_model['ws_pred'],marker='^',edgecolor="black",s= 100,linewidth=0.1,c='deeppink',zorder=2,alpha=0.5)
        plt.annotate("RMSE = "+str(round(root_mean_squared_error(linear_model['ws_pred'],linear_model['ws']),2))+r"$\ \text{ms}{}^{-1}$",(2.9,3.45),size=10)
        
        
        
        # plt.scatter(linear_model['ws'],linear_model['ws_model'],label="RMSE_era5= "+str(round(root_mean_squared_error(linear_model['ws_model'],linear_model['ws']),3))+r"$\text{ms}{}^{-1})$",edgecolor="black",s=25,linewidth=0.5,c='dodgerblue',zorder=2,alpha=0.9)
        # plt.axvline()
        plt.xlabel(r"$u^{obs}\:(\text{ms}{}^{-1})$")
        plt.ylabel(r"$u^{pred}\:(\text{ms}{}^{-1})$")
        # plt.legend(loc='upper left')
        
        # plt.xlim(-2,2)    
        # plt.ylim(-2,2)
        # jaja.dropna(inplace=True)

        ax = plt.gca()
        ax.set_aspect('equal')
        ax.set_facecolor('white')  # Light gray background color
        # ax.yaxis.set_ticks_position('both')
        # ax.xaxis.set_ticks_position('both')
        ax.tick_params(axis='y', labelrotation=90)
        # Show the plot
        plt.xlim(2.9,3.5)
        plt.ylim(2.9,3.5)
        plt.xticks(ticks=np.arange(2.9,3.5,0.1))
        # plt.minorticks_on()
        RMSE_ours.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_pred']))
        RMSE_era.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_model']))
        # Save each plot to a new pagev
        pdf.savefig()
        plt.show()

print(f"Plots saved to {pdf_path}.")

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3453350242.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp':'temp1'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3453350242.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp_model':'temp'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3453350242.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

Plots saved to model_fit_works_one_stations.pdf.


# Results figure 2: Model fit works in all stations

In [5]:

from matplotlib.backends.backend_pdf import PdfPages
# import scienceplots
from sklearn.metrics import root_mean_squared_error,r2_score
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

# Load parameters and preprocess
params = pd.read_csv(r'New_bigtable/Universalparams.csv')
    



# Station names
# names = [
#      'schiaparelli',
# ]

# Open PDF file
pdf_path = "model_fit_works_all_stations.pdf"
params.set_index('glacier_name',inplace=True)
# params.drop(index=['Kennikot','Langenferner'],inplace=True)
RMSE_ours=[]
RMSE_era=[]
params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)
# params_stats.drop(index=['Kennikot','Langenferner'],inplace=True)
with PdfPages(pdf_path) as pdf:
    golden = (1 + 5 ** 0.5) / 2
    size=10
    plt.ioff()
    fig,[ax]=fig_row(1,sharey=False,sharex=False)
    for i, name in enumerate(tqdm(params_stats.index)):
        leave = []
        
        
        
        # Obtaining predicted parameters
        
        S_pred=params_stats['S_fit'].loc[name]
        D_pred=params_stats['D_fit'].loc[name]
        U_pred=params['mean_wind_obs'].loc[name]
        
        se_S_pred=params_stats['se_S_fit'].loc[name]
        se_D_pred=params_stats['se_D_fit'].loc[name]
        se_U_pred=0
        
        A = S_pred
        B = -D_pred * S_pred

        
        # Getting timeseries data
        data_1=pd.read_csv('Universalhourlydatasets/'+name+'.csv')
        params=pd.read_csv('New_bigtable/Universalparams.csv')
        params.set_index('glacier_name',inplace=True)
        lat=params['lat'].loc[name]
        lon=params['lon'].loc[name]
        data_1=data_1.groupby(data_1.index%24).mean()
        linear_model=data_1[['temp_model','ws','temp','ws_model']]
        linear_model.rename(columns={'temp':'temp1'},inplace=True)
        linear_model.rename(columns={'temp_model':'temp'},inplace=True)
        mean_temp_cache=linear_model['temp'].mean()
        linear_model['temp']=linear_model['temp']-linear_model['temp'].mean()
        linear_model['temp1']=linear_model['temp1']-linear_model['temp1'].mean()
        mean_wind_cache=linear_model['ws'].mean()
        linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
        linear_model['dt']=linear_model['temp'].diff()
        linear_model=linear_model.dropna()

        # use the parameters to predict the wind speed
        linear_model['ws_pred']=A*linear_model['temp']+B*linear_model['dt']+U_pred
        linear_model['d_ws_pred']=np.sqrt(((linear_model['temp']-D_pred*linear_model['dt'])**2)*se_S_pred**2+((S_pred*linear_model['dt'])**2)*se_D_pred**2+se_U_pred**2) # u fit error is zero
        linear_model['ws_pred_upper']=linear_model['ws_pred']+linear_model['d_ws_pred']
        linear_model['ws_pred_lower']=linear_model['ws_pred']-linear_model['d_ws_pred']
        
        linear_model['ws']=linear_model['ws']+mean_wind_cache
        linear_model['temp']=linear_model['temp']+mean_temp_cache
        # Plot
        golden = (1 + 5 ** 0.5) / 2
        size=10
            
        ax.axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
        ax.scatter(linear_model['ws'],linear_model['ws_pred'],label="RMSE_fit= "+str(round(root_mean_squared_error(linear_model['ws_pred'],linear_model['ws']),3))+r"$\text{ms}{}^{-1})$",marker='^',edgecolor="black",zorder=-100,alpha=0.5,linewidth=0,s=20)
        
        # plt.scatter(linear_model['ws'],linear_model['ws_model'],label="RMSE_era5= "+str(round(root_mean_squared_error(linear_model['ws_model'],linear_model['ws']),3))+r"$\text{ms}{}^{-1})$",edgecolor="black",s=25,linewidth=0.5,c='dodgerblue',zorder=2,alpha=0.9)
        # plt.axvline()
        ax.set_xlabel(r"${u}^{obs}\:(\text{ms}{}^{-1})$")
        ax.set_ylabel(r"${u}^{pred}\:(\text{ms}{}^{-1})$")
        # plt.xlabel(r"$u_{obs}\:(\text{ms}{}^{-1})$")
        # plt.ylabel(r"$u_{pred}\:(\text{ms}{}^{-1})$")
        
        # plt.xlim(-2,2)    
        # plt.ylim(-2,2)
        # jaja.dropna(inplace=True
        
        ax.set_aspect('equal')
        ax.set_facecolor('white')  # Light gray background color
        # ax.yaxis.set_ticks_position('both')
        # ax.xaxis.set_ticks_position('both')
        # Show the plot
        ax.set_xlim(-0.5,8.5)
        ax.set_ylim(-0.5,8.5)
        ax.set_xticks(ticks=np.arange(0,9,1))
        ax.tick_params(axis='y', labelrotation=90)
        # ax.minorticks_on()
        # plt.tight_layout()
        # ax.set_title('A.',loc='left')
        RMSE_ours.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_pred']))
        RMSE_era.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_model']))
    # plt.grid(True,which='major')
    # plt.annotate(r"RMSE = $0.24\ \text{ms}^{-1}$",(1.5,7),size=10)
    size=11
# RMSE_ours=pd.read_csv("RMSE_ours.csv")fno
    ax1 = fig.add_axes([0.21,0.57,0.21,0.2])
    ax1.hist(RMSE_ours)
    ax1.set_xticks(np.arange(0,0.7,0.2))
    ax1.set_yticks(np.arange(0,7,2))
    ax1.set_xlim(0,0.7)
    ax1.set_ylim(0,8)
    ax1.set_title(r"RMSE (ms${}^{-1}$)")
    ax1.tick_params(axis='y', labelrotation=90)
    ax1.yaxis.set_label_position("right")
    ax1.yaxis.tick_right()

    # plt.legend("Median RMSE="+str(np.round(np.median(RMSE_ours),2))+r"$ms^{-1}$")
    pdf.savefig()
plt.show()
print(f"Plots saved to {pdf_path}.")


  0%|          | 0/28 [00:00<?, ?it/s]C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1076557951.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp':'temp1'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1076557951.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp_model':'temp'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1076557951.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Plots saved to model_fit_works_all_stations.pdf.


In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(RMSE_ours))
print("Mean RMSE:", np.round(np.mean(RMSE_ours),2))
print("Median RMSE:", np.round(np.median(RMSE_ours),2))
print("Inter quartile range:", np.round(np.quantile(RMSE_ours,0.25),2),"-",np.round(np.quantile(RMSE_ours,0.75),2))
print("Full range:", np.round(np.quantile(RMSE_ours,0),2),"-",np.round(np.quantile(RMSE_ours,1),2))

In [ ]:
pd.DataFrame(data=RMSE_ours,index=params.index).plot.bar()
plt.show()

In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(params['mean_wind_obs']))
print("Mean RMSE:", np.round(np.mean(params['mean_wind_obs']),2))
print("Median RMSE:", np.round(np.median(params['mean_wind_obs']),2))
print("Inter quartile range:", np.round(np.quantile(params['mean_wind_obs'],0.25),2),"-",np.round(np.quantile(params['mean_wind_obs'],0.75),2))
print("Full range:", np.round(np.quantile(params['mean_wind_obs'],0),2),"-",np.round(np.quantile(params['mean_wind_obs'],1),2))

In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(params['sensitivity']))
print("Mean RMSE:", np.round(np.mean(params['sensitivity']),2))
print("Median RMSE:", np.round(np.median(params['sensitivity']),2))
print("Inter quartile range:", np.round(np.quantile(params['sensitivity'],0.25),2),"-",np.round(np.quantile(params['sensitivity'],0.75),2))
print("Full range:", np.round(np.quantile(params['sensitivity'],0),2),"-",np.round(np.quantile(params['sensitivity'],1),2))

In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(params['delay']))
print("Mean RMSE:", np.round(np.mean(params['delay']),2))
print("Median RMSE:", np.round(np.median(params['delay']),2))
print("Inter quartile range:", np.round(np.quantile(params['delay'],0.25),2),"-",np.round(np.quantile(params['delay'],0.75),2))
print("Full range:", np.round(np.quantile(params['delay'],0),2),"-",np.round(np.quantile(params['delay'],1),2))

In [ ]:
pd.DataFrame(data=RMSE_ours,index=params.index).sort_values(by=0).plot.bar()
plt.show()

# Figure 4: Combine above two plots

In [6]:
set_plot_style()

from matplotlib.backends.backend_pdf import PdfPages
# import scienceplots
from sklearn.metrics import root_mean_squared_error,r2_score
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
# Load parameters and preprocess
params = pd.read_csv(r'New_bigtable/Universalparams.csv')
    
# Station names
# names = [
#      'schiaparelli',
# ]

# Open PDF file
pdf_path = "model_fit_works_one_stations.pdf"
params.set_index('glacier_name',inplace=True)
RMSE_ours=[]
RMSE_era=[]
params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)
with PdfPages(pdf_path) as pdf:
    for i, name in enumerate(tqdm(['Place3'])):
        leave = []
        
        
        
        # Obtaining predicted parameters
        
        S_pred=params_stats['S_fit'].loc[name]
        D_pred=params_stats['D_fit'].loc[name]
        U_pred=params['mean_wind_obs'].loc[name]
        
        se_S_pred=params_stats['se_S_fit'].loc[name]
        se_D_pred=params_stats['se_D_fit'].loc[name]
        se_U_pred=params_stats['se_U_fit'].loc[name]
        
        A = S_pred
        B = -D_pred * S_pred

        
        
        # Getting timeseries data
        data_1=pd.read_csv('Universalhourlydatasets/'+name+'.csv')
        params=pd.read_csv('New_bigtable/Universalparams.csv')
        params.set_index('glacier_name',inplace=True)
        lat=params['lat'].loc[name]
        lon=params['lon'].loc[name]
        data_1=data_1.groupby(data_1.index%24).mean()
        linear_model=data_1[['temp_model','ws','temp','ws_model']]
        linear_model.rename(columns={'temp':'temp1'},inplace=True)
        linear_model.rename(columns={'temp_model':'temp'},inplace=True)
        mean_temp_cache=linear_model['temp'].mean()
        linear_model['temp']=linear_model['temp']-linear_model['temp'].mean()
        linear_model['temp1']=linear_model['temp1']-linear_model['temp1'].mean()
        mean_wind_cache=linear_model['ws'].mean()
        linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
        linear_model['dt']=linear_model['temp'].diff()
        linear_model=linear_model.dropna()
        station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
        station_data['datetime'] = pd.to_datetime(station_data['datetime'])
        station_data[station_data['ws']<-0.1]=np.nan
        station_data.dropna(inplace=True)
        station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
        station_data['year'] = station_data['datetime'].dt.year
        year_with_max_data = station_data['year'].value_counts().idxmax()
        station_data.set_index('datetime', inplace=True)
        station_data_hourly = station_data
        station_data_hourly.dropna(inplace=True)
        station_data_hourly = station_data_hourly.resample('1h').mean()
        # station_data_hourly=station_data_hourly.interpolate(method='time')
        station_data = station_data_hourly.reset_index()
        error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
        error_linear_model=error_linear_model[1:]
        linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
        linear_model['ddt']=2*linear_model['dtemp']
        
        
        # use the parameters to predict the wind speed
        linear_model['ws_pred']=A*linear_model['temp']+B*linear_model['dt']+U_pred
        linear_model['d_ws_pred']=np.sqrt((se_U_pred**2)+(S_pred)**2*linear_model['dtemp']+(linear_model['temp']-D_pred*linear_model['dt'])**2*se_S_pred**2+(S_pred**2*D_pred**2*linear_model['dtemp']**2)+(S_pred*linear_model['dt'])**2*se_D_pred**2)
        linear_model['ws_pred_upper']=linear_model['ws_pred']+linear_model['d_ws_pred']
        linear_model['ws_pred_lower']=linear_model['ws_pred']-linear_model['d_ws_pred']
        
        linear_model['ws']=linear_model['ws']+mean_wind_cache
        linear_model['temp']=linear_model['temp']+mean_temp_cache
        # Plot
        plt.ioff()
        fig,[ax,ax1,ax3]=fig_row(3,sharey=False,sharex=False)
        ax.text(0.05, 0.95, 'a.', transform=ax.transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
        ax1.text(0.05, 0.95, 'b.', transform=ax1.transAxes,
                fontsize=15, fontweight='bold', va='center', ha='center')
        ax3.text(0.05, 0.95, 'c.', transform=ax3.transAxes,
                fontsize=15, fontweight='bold', va='center', ha='center')

        l1=ax.plot(linear_model['ws'].rolling(1).mean().index,linear_model['ws'].rolling(1).mean(),linewidth=1,color='purple',marker='o',label=r"$u^{obs}$")
        # ax.fill_between(linear_model['ws'].rolling(1).mean().index,linear_model['ws'].rolling(1).mean()+error_linear_model['ws'].rolling(1).mean(),linear_model['ws'].rolling(1).mean()-error_linear_model['ws'].rolling(1).mean(),color='purple',linewidth=0.1,zorder=10,alpha=0.2)
        l2=ax.plot(linear_model['ws_pred'].rolling(1).mean().index,linear_model['ws_pred'].rolling(1).mean(),linewidth=1,color='deeppink',marker='^',linestyle='--',label=r"$u^{pred}$")
        # ax.fill_between(linear_model['ws_pred'].rolling(1).mean().index,linear_model['ws_pred_upper'].rolling(1).mean(),linear_model['ws_pred_lower'].rolling(1).mean(),color='deeppink',linewidth=0.1,zorder=10,alpha=0.3)
        l3=ax.plot(linear_model['ws_model'].rolling(1).mean().index,linear_model['ws_model'].rolling(1).mean(),linewidth=1,color='dodgerblue',marker='s',linestyle='--',label=r"$u^{era}$")
        # ax.fill_between(linear_model['ws_model'].rolling(1).mean().index,linear_model['ws_model'].rolling(1).mean()+error_linear_model['ws_model'].rolling(1).mean(),linear_model['ws_model'].rolling(1).mean()-error_linear_model['ws_model'].rolling(1).mean(),color='dodgerblue',linewidth=0.1,zorder=10,alpha=0.2)
        
        # ax.annotate("RMSE = "+str(round(root_mean_squared_error(linear_model['ws_pred'],linear_model['ws']),2))+r"$\ \text{ms}{}^{-1}$",(8,3.5),size=10)
        # ax.annotate(
        #             "RMSE = " + str(round(root_mean_squared_error(linear_model['ws_pred'], linear_model['ws']), 2)) + r"$\ \text{ms}{}^{-1}$",
        #             xy=(0.05, 0.7),
        #             xycoords='axes fraction',
        #             ha='left'
        #             )
        ax.set_ylabel(r"Wind speed (ms$^{-1}$)")
        
        lns=l1+l2+l3
        labs = [l.get_label() for l in lns]
        ax.legend(lns, labs, loc='upper left', bbox_to_anchor=(0.02, 0.92))
        ax.set_xticks(ticks=np.arange(0,25,6))
        ax.set_xticks(ticks=np.arange(3,25,6),minor='True')

        # ax.set_title(f"$\\bar{{u}}={U_pred:.2f}$, $s={S_pred:.2f} \\pm {se_S_pred:.2f}$, $\\tau={D_pred:.2f} \\pm {se_D_pred:.2f}$")
        ax.set_yticks(ticks=np.arange(0.5,4.1,1),minor='True')
        ax.set_yticks(ticks=np.arange(1,4.1,1))
        ax.set_ylim(0,4.5)
        
        
        plt.yticks(rotation=90)
        ax.set_xlabel("Hour of day in UTC")
        plt.axes(ax1)
        # plt.rcParams["font.family"] = "Helvetica"
        # plt.rcParams['font.size']=20
        # plt.grid(True,zorder=-1)
        # plt.axhline(0,color='black',linewidth=1,linestyle='--')
        
        # plt.errorbar(linear_model['ws'],linear_model['ws_pred'],xerr=error_linear_model['ws'],yerr=linear_model['d_ws_pred'],linewidth=0,elinewidth=0.2,color='deeppink',capsize=2,capthick=0.3)
        plt.scatter(linear_model['ws'],linear_model['ws_pred'],marker='^',edgecolor="black",s= 100,linewidth=0,c='deeppink',zorder=2,alpha=0.8)
        plt.axline((0,0),(1,1),color='black',linewidth=0.3,linestyle='-')
        plt.annotate(
                    "RMSE = " + str(round(root_mean_squared_error(linear_model['ws_pred'], linear_model['ws']), 2)) + r"$\ \text{ms}{}^{-1}$",
                    xy=(0.05, 0.8),
                    xycoords='axes fraction',
                    ha='left'
                    )

        
        # plt.scatter(linear_model['ws'],linear_model['ws_model'],label="RMSE_era5= "+str(round(root_mean_squared_error(linear_model['ws_model'],linear_model['ws']),3))+r"$\text{ms}{}^{-1})$",edgecolor="black",s=25,linewidth=0.5,c='dodgerblue',zorder=2)
        # plt.axvline()
        plt.xlabel(r"$u^{obs}\:(\text{ms}{}^{-1})$")
        plt.ylabel(r"$u^{pred}\:(\text{ms}{}^{-1})$")
        # plt.legend(loc='upper left')
        
        # plt.xlim(-2,2)    
        # plt.ylim(-2,2)
        # jaja.dropna(inplace=True)

        ax = plt.gca()
        ax.set_aspect('equal')
        ax.set_facecolor('white')  # Light gray background color
        # ax.yaxis.set_ticks_position('both')
        # ax.xaxis.set_ticks_position('both')
        ax.tick_params(axis='y', labelrotation=90)
        # Show the plot
        plt.xlim(2.9,3.5)
        plt.ylim(2.9,3.5)
        ax.set_yticks(ticks=np.arange(2.9,3.5,0.2),minor='True')
        ax.set_yticks(ticks=np.arange(3.0,3.45,0.2))
        ax.set_xticks(ticks=np.arange(2.9,3.5,0.2),minor='True')
        ax.set_xticks(ticks=np.arange(3.0,3.45,0.2))
        # plt.minorticks_on()
        RMSE_ours.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_pred']))
        RMSE_era.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_model']))
        # Save each plot to a new pagev
        pdf.savefig()
        plt.show()

print(f"Plots saved to {pdf_path}.")



from matplotlib.backends.backend_pdf import PdfPages
# import scienceplots
from sklearn.metrics import root_mean_squared_error,r2_score
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.linear_model import LinearRegression

# Load parameters and preprocess
params = pd.read_csv(r'New_bigtable/Universalparams.csv')
    



# Station names
# names = [
#      'schiaparelli',
# ]

# Open PDF file
pdf_path = "model_fit_works_all_stations.pdf"
params.set_index('glacier_name',inplace=True)
# params.drop(index=['Kennikot','Langenferner'],inplace=True)
RMSE_ours=[]
RMSE_era=[]
params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)
# params_stats.drop(index=['Kennikot','Langenferner'],inplace=True)
with PdfPages(pdf_path) as pdf:
    golden = (1 + 5 ** 0.5) / 2
    size=10
    plt.ioff()
    # fig,[ax]=fig_row(1,sharey=False,sharex=False)
    ax=ax3
    for i, name in enumerate(tqdm(params_stats.index)):
        leave = []
        
        
        
        # Obtaining predicted parameters
        
        S_pred=params_stats['S_fit'].loc[name]
        D_pred=params_stats['D_fit'].loc[name]
        U_pred=params['mean_wind_obs'].loc[name]
        
        se_S_pred=params_stats['se_S_fit'].loc[name]
        se_D_pred=params_stats['se_D_fit'].loc[name]
        se_U_pred=params_stats['se_U_fit'].loc[name]
        
        A = S_pred
        B = -D_pred * S_pred

        
        # Getting timeseries data
        data_1=pd.read_csv('Universalhourlydatasets/'+name+'.csv')
        params=pd.read_csv('New_bigtable/Universalparams.csv')
        params.set_index('glacier_name',inplace=True)
        lat=params['lat'].loc[name]
        lon=params['lon'].loc[name]
        data_1=data_1.groupby(data_1.index%24).mean()
        linear_model=data_1[['temp_model','ws','temp','ws_model']]
        linear_model.rename(columns={'temp':'temp1'},inplace=True)
        linear_model.rename(columns={'temp_model':'temp'},inplace=True)
        mean_temp_cache=linear_model['temp'].mean()
        linear_model['temp']=linear_model['temp']-linear_model['temp'].mean()
        linear_model['temp1']=linear_model['temp1']-linear_model['temp1'].mean()
        mean_wind_cache=linear_model['ws'].mean()
        linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
        linear_model['dt']=linear_model['temp'].diff()
        linear_model=linear_model.dropna()
        station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
        station_data['datetime'] = pd.to_datetime(station_data['datetime'])
        station_data[station_data['ws']<-0.1]=np.nan
        station_data.dropna(inplace=True)
        station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
        station_data['year'] = station_data['datetime'].dt.year
        year_with_max_data = station_data['year'].value_counts().idxmax()
        station_data.set_index('datetime', inplace=True)
        station_data_hourly = station_data
        station_data_hourly.dropna(inplace=True)
        station_data_hourly = station_data_hourly.resample('1h').mean()
        # station_data_hourly=station_data_hourly.interpolate(method='time')
        station_data = station_data_hourly.reset_index()
        error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
        error_linear_model=error_linear_model[1:]
        linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
        linear_model['ddt']=2*linear_model['dtemp']
        # use the parameters to predict the wind speed
        linear_model['ws_pred']=A*linear_model['temp']+B*linear_model['dt']+U_pred
        linear_model['d_ws_pred']=np.sqrt((se_U_pred**2)+(S_pred)**2*linear_model['dtemp']+(linear_model['temp']-D_pred*linear_model['dt'])**2*se_S_pred**2+(S_pred**2*D_pred**2*linear_model['dtemp']**2)+(S_pred*linear_model['dt'])**2*se_D_pred**2)
        linear_model['ws_pred_upper']=linear_model['ws_pred']+linear_model['d_ws_pred']
        linear_model['ws_pred_lower']=linear_model['ws_pred']-linear_model['d_ws_pred']
        linear_model.to_csv('Universalhourlydatasets/Fittedhourlydatasets/'+name+'.csv')
        
        linear_model['ws']=linear_model['ws']+mean_wind_cache
        linear_model['temp']=linear_model['temp']+mean_temp_cache
        # Plot
        golden = (1 + 5 ** 0.5) / 2
        size=10
            
        ax.axline((0,0),(1,1),color='black',linewidth=0.1,linestyle='-')
        ax.scatter(linear_model['ws'],linear_model['ws_pred'],label="RMSE_fit= "+str(round(root_mean_squared_error(linear_model['ws_pred'],linear_model['ws']),3))+r"$\text{ms}{}^{-1})$",marker='^',edgecolor="black",zorder=-100,linewidth=0.1,s=20,alpha=0.6)
        # ax.errorbar(linear_model['ws'],linear_model['ws_pred'],xerr=error_linear_model['ws'],yerr=linear_model['d_ws_pred'],linewidth=0,elinewidth=0.2,capsize=1,capthick=0.2)
        # plt.scatter(linear_model['ws'],linear_model['ws_model'],label="RMSE_era5= "+str(round(root_mean_squared_error(linear_model['ws_model'],linear_model['ws']),3))+r"$\text{ms}{}^{-1})$",edgecolor="black",s=25,linewidth=0.5,c='dodgerblue',zorder=2)
        # plt.axvline()
        ax.set_xlabel(r"${u}^{obs}\:(\text{ms}{}^{-1})$")
        ax.set_ylabel(r"${u}^{pred}\:(\text{ms}{}^{-1})$")
        # plt.xlabel(r"$u_{obs}\:(\text{ms}{}^{-1})$")
        # plt.ylabel(r"$u_{pred}\:(\text{ms}{}^{-1})$")
        
        # plt.xlim(-2,2)    
        # plt.ylim(-2,2)
        # jaja.dropna(inplace=True
        
        ax.set_aspect('equal')
        ax.set_facecolor('white')  # Light gray background color
        # ax.yaxis.set_ticks_position('both')
        # ax.xaxis.set_ticks_position('both')
        # Show the plot
        ax.set_xlim(-0.5,9.5)
        ax.set_ylim(-0.5,9.5)
        ax.set_yticks(ticks=np.arange(0,10,2))
        ax.set_yticks(ticks=np.arange(1,10,2),minor='True')
        ax.set_xticks(ticks=np.arange(0,10,2))
        ax.set_xticks(ticks=np.arange(1,10  ,2),minor='True')
        ax.tick_params(axis='y', labelrotation=90)
        # ax.minorticks_on()
        # plt.tight_layout()
        # ax.set_title('A.',loc='left')
        RMSE_ours.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_pred']))
        RMSE_era.append(root_mean_squared_error(linear_model['ws'], linear_model['ws_model']))
    # plt.grid(True,which='major')
    # plt.annotate(r"RMSE = $0.24\ \text{ms}^{-1}$",(1.5,7),size=10)
    size=11
# RMSE_ours=pd.read_csv("RMSE_ours.csv")fno
    ax1 = fig.add_axes([0.72,0.57,0.08,0.2])
    ax1.hist(RMSE_ours,color='pink')
    ax1.set_xticks(np.arange(0,0.7,0.2))
    ax1.set_yticks(np.arange(0,7,2))
    ax1.set_xlim(0,0.7)
    ax1.set_ylim(0,8)
    ax1.set_xlabel(r"RMSE")
    ax1.tick_params(axis='y', labelrotation=90)
    ax1.yaxis.set_label_position("right")
    ax1.yaxis.tick_right()

    # plt.legend("Median RMSE="+str(np.round(np.median(RMSE_ours),2))+r"$ms^{-1}$")
    pdf.savefig(bbox_inches='tight')
plt.show()
print(f"Plots saved to {pdf_path}.")


  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1139781323.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp':'temp1'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1139781323.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp_model':'temp'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1139781323.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

Plots saved to model_fit_works_one_stations.pdf.


  0%|          | 0/28 [00:00<?, ?it/s]C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1139781323.py:246: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp':'temp1'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1139781323.py:247: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linear_model.rename(columns={'temp_model':'temp'},inplace=True)
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\1139781323.py:249: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

Plots saved to model_fit_works_all_stations.pdf.


In [ ]:
params.index

# Supplementary Fig. S2 Model coefficient distribution

In [7]:
fig, ax = fig_row(3)
plt.rcParams['font.size']=15
# Plot mean winds
ax[0].hist(params['mean_wind_obs'], bins=10, color='lightpink',rwidth=1,zorder=100)
# ax[0].set_title('Mean Winds')
ax[0].set_xlabel(r'$\bar{u}$'+r" $(\text{ms}{}^{-1})$")
ax[0].set_ylabel('Frequency')
# Plot sensitivities
ax[0].tick_params(axis='y', rotation=90)
ax[1].tick_params(axis='y', rotation=90)
ax[2].tick_params(axis='y', rotation=90)
#make tick labels at 0,2,4,6,8,10
ax[1].hist(params['sensitivity'], bins=10, color='lightpink',rwidth=1,zorder=100)
# ax[1].set_title('Sensitivities')
ax[1].set_xlabel(r'$s$'+r" $(\text{ms}{}^{-1}{^\circ \text{C}}^{-1})$")
ax[1].set_ylabel('')

# Plot delays
ax[2].hist(params['delay'], bins=10, color='lightpink', rwidth=1,zorder=100)
# ax[2].set_title('Delays')
ax[2].set_xlabel(r'$\tau$ (hr)')
ax[2].set_ylabel('')
ax[0].set_yticks(np.arange(0, 8, 2))
ax[1].set_yticks(np.arange(0, 14, 4))
ax[2].set_yticks(np.arange(0,   15, 4))

plt.subplots_adjust(hspace=0.1,wspace=0.1)
plt.savefig("model_coefficient_distribution.pdf")
plt.show()

# Figure 5: Mean wind parameterisation

In [8]:

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import root_mean_squared_error,r2_score
import seaborn as sns

params_stats=pd.read_csv('params_stats.csv')
# Leave one out cross validation

rmse_lres=[]
rmse_lreg=[]
rmse_era5=[]

yodata=[]
ypdata=[]
ypldata=[]
yedata=[]

params=pd.read_csv(r'New_bigtable/Universalparams.csv')


# params=params[~params['glacier_name'].isin(['Otemma','Langenferner','Weissee','Corbassiere','Vernatferner','Balcony'])]
params.set_index('glacier_name',inplace=True)
# params.drop(index='Kennikot',inplace=True)

# x_data_mean_wind=params[['relief_5km', 'stddev_1km', 'Slope_10km', 'rgi_slope', 'rel_Z', 'rel_distancefromhead', 'rel_distancefromcenterline', 'aspect_ratio_relief_1']]
# x_data_mean_wind=params[['relief_5km', 'stddev_1km', 'Slope_10km', 'rgi_slope', 'aspect_ratio_stddev_1']]
# x_data_mean_wind=params[['relief_5km', 'stddev_1km', 'rgi_slope', 'aspect_ratio_stddev_1']]
x_data_mean_wind=params[[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']]
y_data_mean_wind=params['mean_wind_obs']
x_data_sensitivity=params[[ 'station_z', 'Z_mean_10km']]
y_data_sensitivity=params['sensitivity']
y_data_linsensitivity=params[['sensitivity']]
x_data_delay=params[[ 'Slope_100m']]
y_data_delay=params['delay']


params['RGI_region']=[s[0:17] for s in params['Glacier_ID']]

RMSE_ours=[]
RMSE_era5=[]
same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index
for i,name in enumerate(params.index):
    
    # sensitivity check, to see if the fit is dependent on regions.
    
    rgi_region_station=params['RGI_region'].loc[name]
    # print(rgi_region_station)
    same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index

    # print(same_region_stations)
    same_region_stations=[]
    # if i==1:
    #     continue
    # mean wind prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    
    leave=np.append(same_region_stations,["Kennikot"])
    model=Lr.fit(X=x_data_mean_wind.drop(index=leave),y=y_data_mean_wind.drop(index=leave))
    
    x_validate_mean_wind=x_data_mean_wind.loc[[name]]
    
    if x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]>40:
        x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
        print("Ad-hoc threshold applied")    
    Mean_wind=model.predict(x_validate_mean_wind)





    A=0
    B=0


    D_reg=0
    A_reg=0
    B_reg=0

    # Time series
    # name=names[i]
    station_data=pd.read_csv('Universalhourlydatasets//'+name+'.csv')
    station_data['datetime']=pd.to_datetime(station_data['datetime'])
    linear_model=station_data.copy()
    linear_model['temp_model']=linear_model['temp_model']-linear_model['temp_model'].mean()
    linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
    linear_model['ws_model']=linear_model['ws_model']-linear_model['ws_model'].mean()

    station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
    station_data['datetime'] = pd.to_datetime(station_data['datetime'])
    station_data[station_data['ws']<-0.1]=np.nan
    station_data.dropna(inplace=True)
    station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
    station_data['year'] = station_data['datetime'].dt.year
    year_with_max_data = station_data['year'].value_counts().idxmax()
    station_data.set_index('datetime', inplace=True)
    station_data_hourly = station_data
    station_data_hourly.dropna(inplace=True)
    station_data_hourly = station_data_hourly.resample('1h').mean()
    # station_data_hourly=station_data_hourly.interpolate(method='time')
    station_data = station_data_hourly.reset_index()
    error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
    error_linear_model=error_linear_model[1:]
    linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
    linear_model['ddt']=2*linear_model['dtemp']

    lres_wind_speed=np.array(((A*linear_model['temp_model']+B*linear_model['temp_model'].diff())).replace(np.nan,0))*0+Mean_wind
    observed_wind_speed=station_data['ws'].values*0+station_data['ws'].mean()
    era5_wind_speed=station_data['ws_model'].values*0+station_data['ws_model'].mean()

    station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
    station_data['datetime'] = pd.to_datetime(station_data['datetime'])
    station_data[station_data['ws']<-0.1]=np.nan
    station_data.dropna(inplace=True)
    station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
    station_data['year'] = station_data['datetime'].dt.year
    year_with_max_data = station_data['year'].value_counts().idxmax()
    station_data.set_index('datetime', inplace=True)
    station_data_hourly = station_data
    station_data_hourly.dropna(inplace=True)
    station_data_hourly = station_data_hourly.resample('1h').mean()
    # station_data_hourly=station_data_hourly.interpolate(method='time')
    station_data = station_data_hourly.reset_index()
    error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
    error_linear_model=error_linear_model[1:]
    linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
    linear_model['ddt']=2*linear_model['dtemp']
    
    # if not (i==3):
    for l in range(1):
        yodata.append(observed_wind_speed[l])
        ypdata.append(lres_wind_speed[l])
        yedata.append(era5_wind_speed[l])
        RMSE_era5.append(np.abs(observed_wind_speed[l]-era5_wind_speed[l]))
        RMSE_ours.append(np.abs(observed_wind_speed[l]-lres_wind_speed[l]))
    
    
    
    
jaja=pd.DataFrame()

jaja['u_era5']=yedata#-jaja['u_obs']s
jaja['u_S']=ypdata#-jaja['u_obs']
jaja['u_SD']=ypdata
jaja['u_obs']=yodata
jaja['glacier_number']=np.arange(len(yedata))



pd.DataFrame(RMSE_era5).to_csv("RMSE_era5.csv")
pd.DataFrame(RMSE_ours).to_csv("RMSE_ours.csv")
pd.DataFrame(data={'upred':jaja['u_SD'].values},index=params.index).to_csv("Mean_wind_prediction.csv")
# import scienceplots

# plt.style.use(['science','scatter'])
# plt.rcParams["font.family"] = "Helvetica"
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
# plt.rcParams["axes.titleweight"] = "bold"

fig,ax=fig_row(2,sharey=False)
plt.axes(ax[1])
# plt.rcParams['font.size']=20
# plt.grid(True,zorder=-1)
# plt.axhline(0,color='black',linewidth=1,linestyle='--')
plt.axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
# plt.vlines(jaja['u_obs'],jaja['u_era5'],jaja['u_SD'],linewidth=0.4,linestyle='--',color='gray',zorder=1)
# sns.regplot(data=jaja, x='u_obs',y='their_res',ci=99)
import matplotlib
cmap=matplotlib.colormaps['prism']
ax[0].text(0.05, 0.95, 'a.', transform=ax[0].transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
ax[1].text(0.05, 0.95, 'b.', transform=ax[1].transAxes,
        fontsize=15, fontweight='bold', va='center', ha='center')


import seaborn as sns
# sns.kdeplot(data=jaja, x='u_obs',y='u_era5',ax=plt.gca(),color='dodgerblue',fill=True,levels=20,alpha=0.5,thresh=0.3,linewidth=0.5)
# sns.kdeplot(data=jaja, x='u_obs',y='u_SD',ax=plt.gca(),color='deeppink',fill=True,levels=20,alpha=0.5,thresh=0.3,linewidth=0.5)

plt.scatter(jaja['u_obs'],jaja['u_SD'],label="Model",marker='^',edgecolor="black",s= 50,linewidth=0,c='deeppink',zorder=2,alpha=0.8)
betas=pd.read_csv('betas.csv')



# plt.errorbar(jaja['u_obs'],jaja['u_SD'],xerr=params_stats['se_U_fit'],yerr=params_stats['se_U_pred'],linewidth=0,elinewidth=0.2,color='deeppink',capsize=1,capthick=0.2)
text1="RMSE = "+str(round(np.mean(abs(jaja['u_obs']-jaja['u_SD'])),2))+r"$\ \text{ms}{}^{-1}$"
text2="RMSE = "+str(round(np.mean(abs(jaja['u_obs']-jaja['u_era5'])),2))+r"$\ \text{ms}{}^{-1}$"
# plt.annotate(text1,(3,5.1),color='deeppink',size=10,ha='center')
plt.scatter(jaja['u_obs'],jaja['u_era5'],label="ERA5L",edgecolor="black",s=50,linewidth=0,c='dodgerblue',zorder=1,alpha=0.8)
# plt.annotate(text2,(3,4.8),color='dodgerblue',size=10,ha='center')




# plt.axvline()
plt.xlabel(r"$\bar{u}^{obs}\:(\text{ms}{}^{-1})$")
plt.ylabel(r"$\bar{u}^{pred}\:(\text{ms}{}^{-1})$")
plt.legend(loc='upper left',bbox_to_anchor=(0, 0.93))

# plt.xlim(-2,2)
# plt.ylim(-2,2)
# jaja.dropna(inplace=True)

ax[1].set_facecolor('white')  # Light gray background color
# ax[1].yaxis.set_ticks_position('both')
# ax[1].xaxis.set_ticks_position('both')
ax[1].set_yticks(np.arange(0,6,2))
ax[1].set_yticks(np.arange(1,6,2),minor=True)
ax[1].set_xticks(np.arange(0,6,2))
ax[1].set_xticks(np.arange(1,6,2),minor=True)
ax[1].tick_params(axis='y', labelrotation=90)
ax[1].annotate(
                    text1,
                    xy=(0.05, 0.75),
                    xycoords='axes fraction',
                    ha='left',
                    color='deeppink'
                    )
ax[1].annotate(
                    text2,
                    xy=(0.05, 0.69),
                    xycoords='axes fraction',
                    ha='left',
                    color='dodgerblue'
                    )
# Show the plot
plt.xlim(0.5,5.9)
plt.ylim(0.5,5.9)



sc=ax[0].scatter(params['relief_5km'],params['mean_wind_obs'],c='purple',s=50,alpha=0.6,linewidth=0)

# ax[0].errorbar(params['relief_5km'],params['mean_wind_obs'],yerr=params_stats['se_U_fit'],linewidth=0,elinewidth=0.2,capsize=1,capthick=0.2,color='purple')
# cbar=plt.colorbar(sc,ax=ax[1])
# cbar.set_label("Relief of 1-km buffer")
ax[0].set_xlabel(r"$\Delta Z_5$ (m)")
ax[0].set_ylabel(r"$\bar{u}^{obs}\:(\text{ms}{}^{-1})$")
ax[0].set_ylim(0,5.5)
ax[0].set_yticks(np.arange(0,5,2))
ax[0].set_yticks(np.arange(1,5,2),minor=True)
ax[0].set_xticks(np.arange(2000,4000,1000))
ax[0].set_xticks(np.arange(1500,4000,1000),minor=True)

# sc=ax[2].scatter(params['relief_1km'],params['mean_wind_obs'],c=params['relief_5km'],edgecolor='0.1',s=50,cmap='coolwarm')

# cbar=plt.colorbar(sc,ax=ax[2])
# cbar.set_label("Relief of 5-km buffer")
# ax[2].set_xlabel(r"Relief of 1-km buffer")
# ax[2].set_ylabel(r"$\text{Mean windspeed observed}\:(\text{ms}{}^{-1})$")


# ax[1].set_ylabel(r"$\bar{u}_{obs}\:(\text{ms}{}^{-1})$")
plt.show()
# plt.minorticks_on()
plt.tight_layout()
plt.savefig("mean_winds_parameterisation.pdf")
# plt.close()         


###########################################%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# Blind ERA5L lin regression
###########################################%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


# plt.figure(figsize=(9,8))
# plt.rcParams['font.size']=15
# plt.grid(True,zorder=-1)
# jaja['their_res']=jaja['our_linres']
# jaja['our_linres']=jaja['our_res']
# # plt.axhline(0,color='black',linewidth=1,linestyle='--')
# plt.axline((0,0),(1,1),color='black',linewidth=2,linestyle='-')
# plt.vlines(jaja['u_obs'],jaja['our_linres'],jaja['their_res'],linewidth=0.8,linestyle='--',color='navy',zorder=1)
# plt.scatter(jaja['u_obs'],jaja['their_res'],label='ERA_5+lin_regression, n=15',edgecolor="black",s=50,linewidth=0.2,c='navy',zorder=2)
# # sns.regplot(data=jaja, x='u_obs',y='their_res',ci=99)
# import matplotlib
# cmap=matplotlib.colormaps['prism']
# plt.scatter(jaja['u_obs'],jaja['our_linres'],label="ERA_5+lin_response, n=15",edgecolor='black',linewidth=0.3,s=30,c='deeppink',marker='^',zorder=2,cmap='Paired')
# # plt.axvline()
# plt.xlabel(r"$wind_{obs}\:(m/s)$")
# plt.ylabel(r"$wind_{model}\:(m/s)$")
# plt.legend(loc='upper left')

# # plt.xlim(-2,2)
# # plt.ylim(-2,2)
# # jaja.dropna(inplace=True)
# plt.text(1,3.5,"RMSE="+str(round(root_mean_squared_error(jaja['u_obs'],jaja['their_res']),3))+"\nR^2_adj="+str(round(r2_score(jaja['u_obs'],jaja['their_res']),3)),color='navy',fontdict={'size':15})
# plt.text(1,3,"RMSE="+str(round(root_mean_squared_error(jaja['u_obs'],jaja['our_linres']),3))+"\nR^2_adj="+str(round(r2_score(jaja['u_obs'],jaja['our_linres']),3)),color='deeppink',fontdict={'size':15})

# ax = plt.gca()
# ax.set_facecolor('white')  # Light gray background color
# plt.xlim(0,5)
# plt.ylim(0,5)
# # Show the plot
# plt.show()
# plt.tight_layout()
# plt.savefig("Blind_era5_linregr.pdf")
# # plt.close()



C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3939075192.py:101: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3939075192.py:123: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3939075192.py:75: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the 

Ad-hoc threshold applied


In [ ]:
np.mean(jaja['u_era5']-jaja['u_obs'])

In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(RMSE_ours))
print("Mean RMSE:", np.round(np.mean(RMSE_ours),2))
print("Median RMSE:", np.round(np.median(RMSE_ours),2))
print("Inter quartile range:", np.round(np.quantile(RMSE_ours,0.25),2),"-",np.round(np.quantile(RMSE_ours,0.75),2))
print("Full range:", np.round(np.quantile(RMSE_ours,0),2),"-",np.round(np.quantile(RMSE_ours,1),2))

In [ ]:
mean_wind_rmse_df=pd.DataFrame(data=RMSE_ours,index=params.index)
mean_wind_rmse_df['distancefromhead']=params['mean_abs_along_valley%']

In [ ]:
sns.jointplot(data=mean_wind_rmse_df,x='distancefromhead',y=0)
plt.show()

In [ ]:
pd.DataFrame(data=RMSE_ours,index=params.index).sort_values(by=0).plot.bar()
plt.show()

In [ ]:
# print("Aspect ratio contributes ",betas['Beta'][3]*params['aspect_ratio_stddev_1'].loc['Kennikot'], "among",(betas['Beta'][3]*params['aspect_ratio_stddev_1'].loc['Kennikot']+betas['Beta'][2]*params['relief_5km'].loc['Kennikot']+betas['Beta'][1]*params['relief_1km'].loc['Kennikot']+betas['Beta'][0]))

# Figure 6: Diurnal wind parameterisation

In [9]:
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error,r2_score
import seaborn as sns

#  

# names=[
# 'arolla',
# 'langenferner',
# 'schiaparelli',
# 'Trakarding',
# 'yala',
# 'lirung',
# 'satopanth',
# 'wm2',
# 'pm3',
# 'pm2',
# 'pm1',
# 'djankuat',
# 'chotashigri',
# 'wm1',
# 'drangdrung',
# 'pyramid',
# 'bishop',
# # 'weisssee',
# 'hef',
# # 'namche',
# # 'balcony',
# 'camp2',
# # 'pm4',
# # 'southcol',
# # 'Denalipass',
# 'kalapathar',
# 'bellavista',
# 'millwitzkeez',
# 'venedegerkees',
# 'vernatferner',
# 'otemma',
# 'corbassiere',
# 'kennicot'
# ]
# Leave one out cross validation

rmse_lres=[]
rmse_lreg=[]
rmse_era5=[]

yodata=[]
ypdata=[]
ypldata=[]
yedata=[]
yperror=[]
yoerror=[]
params=pd.read_csv(r'New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)
params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)

# params.drop(index='Kennikot',inplace=True)
# params.drop(index='Langenferner',inplace=True)
names=params.index

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['rel_distancefromterminus','stddev_1']]

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromterminus']]
# y_data_delay=params['delay']

# After choosing best predictor 
x_data_mean_wind=params[[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']]
y_data_mean_wind=params['mean_wind_obs']
x_data_sensitivity=params[[ 'station_z', 'Z_mean_10km']]
# x_data_sensitivity=params[[ 'Z_mean_10km']]
y_data_sensitivity=params['sensitivity']
y_data_linsensitivity=params[['sensitivity']]
x_data_delay=params[[ 'Slope_100m']]
# x_data_delay=params[[ 'glacier_length']]
y_data_delay=params['delay']


# x_data_mean_wind=params[['channel_width_at_station', 'stddev_10', 'T_range_continentality']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['Distance from head','glacier_area']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromhead']]
# y_data_delay=params['delay']
names=list(names)
# for outlier in ['Otemma','Langenferner','Weissee','Corbassiere','Vernatferner','Balcony','Kennikot']:
#     names.remove(outlier)
RMSE_ours=[]
RMSE_era=[]
R2_ours=[]
R2_era=[]
range_ours=[]
range_act=[]
range_era=[]
params['RGI_region']=[s[0:17] for s in params['Glacier_ID']]
same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index


# runfor=same_region_stations
runfor=names


for i,name in enumerate(runfor):
    # if i==1:
    #     continue
    # mean wind prediction
    
    same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index
    same_region_stations=[]
    
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,['Kennikot'])
    model=Lr.fit(X=x_data_mean_wind.drop(index=leave),y=y_data_mean_wind.drop(index=leave))
    x_validate_mean_wind=x_data_mean_wind.loc[[name]]
    Mean_wind=model.predict(x_validate_mean_wind)


    # sensitivity prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_sensitivity.drop(index=leave))
    x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    S=model.predict(x_validate_sensitivity)

    # linear sensitivity prediction
    # from sklearn.linear_model import LinearRegression
    # Lr=LinearRegression()
    # leave=[name]
    # model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_linsensitivity.drop(index=leave))
    # x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    # S_reg=model.predict(x_validate_sensitivity)

    # delay prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_delay.drop(index=leave),y=y_data_delay.drop(index=leave))
    x_validate_delay=x_data_delay.loc[[name]]
    D=model.predict(x_validate_delay)   


    A=S
    B=-D*S


    D_reg=0
    A_reg=0
    B_reg=0

    # Time series
    name=names[i]
    station_data=pd.read_csv('Universalhourlydatasets//'+name+'.csv')
    station_data['datetime']=pd.to_datetime(station_data['datetime'])
    linear_model=station_data.copy()
    linear_model['temp_model']=linear_model['temp_model']-linear_model['temp_model'].mean()
    linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
    linear_model['ws_model']=linear_model['ws_model']-linear_model['ws_model'].mean()

    lres_wind_speed=np.array(((A*linear_model['temp_model']+B*linear_model['temp_model'].diff())).replace(np.nan,0))+Mean_wind*0
    lreg_wind_speed=station_data['ws'].values*0
    observed_wind_speed=station_data['ws'].values-station_data['ws'].mean()*1
    era5_wind_speed=station_data['ws_model'].values-station_data['ws_model'].mean()*1

    linear_model['ws_pred']=lres_wind_speed
    linear_model.to_csv('Universalhourlydatasets/Predictedhourlydatasets/'+name+'.csv')
    S_pred=params_stats['S_pred'].loc[name]
    D_pred=params_stats['D_pred'].loc[name]
    U_pred=params_stats['U_pred'].loc[name]
    
    se_S_pred=params_stats['se_S_pred'].loc[name]
    se_D_pred=params_stats['se_D_pred'].loc[name]
    se_U_pred=params_stats['se_U_pred'].loc[name]
    station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
    station_data['datetime'] = pd.to_datetime(station_data['datetime'])
    station_data[station_data['ws']<-0.1]=np.nan
    station_data.dropna(inplace=True)
    station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
    station_data['year'] = station_data['datetime'].dt.year
    year_with_max_data = station_data['year'].value_counts().idxmax()
    station_data.set_index('datetime', inplace=True)
    station_data_hourly = station_data
    station_data_hourly.dropna(inplace=True)
    station_data_hourly = station_data_hourly.resample('1h').mean()
    # station_data_hourly=station_data_hourly.interpolate(method='time')
    station_data = station_data_hourly.reset_index()
    error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
    error_linear_model=error_linear_model[1:]
    linear_model['dt']=linear_model['temp'].diff()
    linear_model=linear_model.dropna()
    linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
    linear_model['ddt']=2*linear_model['dtemp']
    linear_model['d_ws_pred']=np.sqrt((se_U_pred**2)+(S_pred)**2*linear_model['dtemp']+(linear_model['temp']-D_pred*linear_model['dt'])**2*se_S_pred**2+(S_pred**2*D_pred**2*linear_model['dtemp']**2)+(S_pred*linear_model['dt'])**2*se_D_pred**2)
        
    
    # if not (i==3):
    for l in range(23):
        yodata.append(observed_wind_speed[l])
        ypdata.append(lres_wind_speed[l])
        ypldata.append(lreg_wind_speed[l])
        yedata.append(era5_wind_speed[l])
        yperror.append(linear_model['d_ws_pred'].iloc[l])
        yoerror.append(error_linear_model['ws'].iloc[l])
    RMSE_ours.append(root_mean_squared_error(observed_wind_speed,lres_wind_speed))
    RMSE_era.append(root_mean_squared_error(observed_wind_speed, era5_wind_speed))
    R2_ours.append(r2_score(observed_wind_speed,lres_wind_speed))
    R2_era.append(r2_score(observed_wind_speed,era5_wind_speed))
    range_act.append(np.max(observed_wind_speed)-np.min(observed_wind_speed))
    range_ours.append(np.max(lres_wind_speed)-np.min(lres_wind_speed))
    range_era.append(np.max(era5_wind_speed)-np.min(era5_wind_speed))
jaja=pd.DataFrame()

jaja['u_era5']=yedata#-jaja['u_obs']
jaja['u_S']=ypldata#-jaja['u_obs']
jaja['u_SD']=ypdata
jaja['u_obs']=yodata
jaja['u_oerror']=yoerror
jaja['u_error']=yperror
jaja['glacier_number']=np.arange(23*len(runfor))//23

pd.DataFrame(RMSE_ours).to_csv("RMSE_ours.csv")
pd.DataFrame(RMSE_era).to_csv("RMSE_era5.csv")





fig = plt.figure(figsize=(9, 5.5))
gs = gridspec.GridSpec(2, 2, width_ratios=[1.3, 3], height_ratios=[1, 1])  # Height ratio helps with squares
plt.subplots_adjust(hspace=0.4)
# Left column (two small square plots)
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0])

# Right column (big rectangular plot)
ax3 = fig.add_subplot(gs[:, 1])

# Set titles
ax1.text(0.08, 0.92, 'a.', transform=ax1.transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
ax2.text(0.08, 0.92, 'b.', transform=ax2.transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
ax3.text(0.05, 0.95, 'c.', transform=ax3.transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')



ax=[ax3,ax2,ax1]
# fig.subplots_adjust(wspace=0.2)

ax[0].axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
import seaborn as sns
# sns.kdeplot(data=jaja, x='u_obs',y='u_era5',ax=plt.gca(),color='lightskyblue',fill=True,levels=20,alpha=0.5,thresh=0.4,linewidth=0.5)
# sns.kdeplot(data=jaja, x='u_obs',y='u_SD',ax=plt.gca(),color='deeppink',fill=True,levels=20,alpha=0.5,thresh=0.3,linewidth=0.5)

ax[0].scatter(jaja['u_obs'],jaja['u_SD'],label="Model",marker='^',edgecolor="black",c='deeppink',zorder=2,alpha=0.8,linewidth=0,s=25)
# ax[0].errorbar(jaja['u_obs'],jaja['u_SD'],xerr=jaja['u_oerror'],yerr=jaja['u_error'],linewidth=0,elinewidth=0.1,color='deeppink',capsize=1,capthick=0.2,alpha=0.4)
text1="RMSE = "+str(round(np.mean(RMSE_ours),2))+r" $\text{ms}{}^{-1}$"
ax[0].scatter(jaja['u_obs'],jaja['u_era5'],label="ERA5L",edgecolor="black",c='dodgerblue',zorder=1,alpha=0.8,linewidth=0,s=25)
text2="RMSE = "+str(round(np.mean(RMSE_era),2))+r" $\text{ms}{}^{-1}$"
# plt.annotate(text1,(-1,1.3),ha='center',color='deeppink')
# plt.annotate(text2,(-1,1.1),ha='center',color='dodgerblue')
ax[0].text(0.05, 0.8, text1, transform=ax[0].transAxes,
         fontsize=10, fontweight='regular', va='center', ha='left',color='deeppink')
ax[0].text(0.05, 0.75, text2, transform=ax[0].transAxes,
         fontsize=10, fontweight='regular', va='center', ha='left',color='dodgerblue')


ax[0].set_aspect('equal')
ax[0].set_xlabel(r"$u_d^{obs}\:(\text{ms}{}^{-1})$")
ax[0].set_ylabel(r"$u_d^{pred}\:(\text{ms}{}^{-1})$")
ax[0].legend(loc='upper left',bbox_to_anchor=(0.0,0.93))
ax[0].set_xlim(-2,2)
ax[0].set_ylim(-2,2)
# jaja.dropna(inplace=True)
ax[0].set_facecolor('white')  # Light gray background color
# ax[0].yaxis.set_ticks_position('both')
# ax[0].xaxis.set_ticks_position('both')
ax[0].tick_params(axis='y', labelrotation=90)
ax[0].set_xticks(np.arange(-2,2.1,1))
ax[0].set_xticks(np.arange(-1.5,2,1),minor=True)
ax[0].set_yticks(np.arange(-1,1.9,1))
ax[0].set_yticks(np.arange(-1.5,2,1),minor=True)


# Show the plot
# plt.xlim(0,6)
# plt.ylim(0,6)
# ax[0].minorticks_on()

sc=ax[1].scatter(params['Z_mean_10km'],params['sensitivity'],c='purple',s=50,linewidth=0,alpha=0.6)
# ax[1].errorbar(params['Z_mean_10km'],params['sensitivity'],yerr=params_stats['se_S_fit'],linewidth=0,elinewidth=0.2,color='purple',capsize=1,capthick=0.2)

# cbar=plt.colorbar(sc,ax=ax[1])
# cbar.set_label("Normalised station elevation")
ax[1].set_xlabel(r"$\bar{Z}_{10}$ (m)")
ax[1].set_ylabel(r"$s\:(\text{ms}{}^{-1}{ ^{\circ}\text{C} }^{-1})$")
ax[1].tick_params(axis='y', labelrotation=90)
ax[1].set_ylim(-0.1,1.2)
ax[1].set_yticks(np.arange(0,1.2,0.4))
ax[1].set_yticks(np.arange(0.2,1.2,0.4),minor=True)
ax[1].set_xticks(np.arange(1000,7000,2000),minor=True)

sc=ax[2].scatter(params['Slope_100m'],params['delay'],c='purple',s=50,linewidth=0,alpha=0.6)
# ax[2].errorbar(params['Slope_100m'],params['delay'],yerr=params_stats['se_D_fit'],linewidth=0,elinewidth=0.2,color='purple',capsize=1,capthick=0.2)
# cbar=plt.colorbar(sc,ax=ax[1])
# cbar.set_label("Normalised station elevation")
ax[2].set_xlabel(r"$S_{0.1}$")
ax[2].set_ylabel(r"$\tau\:(\text{hr})$")
ax[2].tick_params(axis='y', labelrotation=90)
ax[2].set_ylim(0,8)
ax[2].set_yticks(np.arange(0,7.9,2))
ax[2].set_yticks(np.arange(-1,8,2),minor=True)
ax[2].set_xlim(-0.1,0.91)
ax[2].set_xticks(np.arange(0,0.9,0.2))
ax[2].set_xticks(np.arange(0.1,0.8,0.2),minor=True)

fig.subplots_adjust(wspace=0.2)

plt.savefig("diurnal_winds_parameterisation.pdf")
plt.show()
# plt.close()



C:\Users\krish\AppData\Local\Temp\ipykernel_14908\4182136457.py:201: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


In [ ]:
plt.plot(pd.DataFrame(RMSE_era).cumsum(),label='ERA5L')
plt.plot(pd.DataFrame(RMSE_ours).cumsum(),label="ours")
plt.show()
plt.legend()

In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(range_act))
print("Mean RMSE:", np.round(np.mean(range_act),2))
print("Median RMSE:", np.round(np.median(range_act),2))
print("Inter quartile range:", np.round(np.quantile(range_act,0.25),2),"-",np.round(np.quantile(range_act,0.75),2))
print("Full range:", np.round(np.quantile(range_act,0),2),"-",np.round(np.quantile(range_act,1),2))

In [ ]:
params.index[7]

# Figure 7: Full model prediction

In [11]:

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error,r2_score
import seaborn as sns

 
set_plot_style()

# names=[
# 'arolla',
# 'langenferner',
# 'schiaparelli',
# 'Trakarding',
# 'yala',
# 'lirung',
# 'satopanth',
# 'wm2',
# 'pm3',
# 'pm2',
# 'pm1',
# 'djankuat',
# 'chotashigri',
# 'wm1',
# 'drangdrung',
# 'pyramid',
# 'bishop',
# # 'weisssee',
# 'hef',
# # 'namche',
# # 'balcony',
# 'camp2',
# # 'pm4',
# # 'southcol',
# # 'Denalipass',
# 'kalapathar',
# 'bellavista',
# 'millwitzkeez',
# 'venedegerkees',
# 'vernatferner',
# 'otemma',
# 'corbassiere',
# 'kennicot'
# ]
# Leave one out cross validation


params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)

rmse_lres=[]
rmse_lreg=[]
rmse_era5=[]

yodata=[]
ypdata=[]
ypldata=[]
yedata=[]
yperror=[]
yoerror=[]

params=pd.read_csv(r'New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)

# params.drop(index='Kennikot',inplace=True)
# params.drop(index='Langenferner',inplace=True)
names=params.index

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['rel_distancefromterminus','stddev_1']]

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromterminus']]
# y_data_delay=params['delay']

# After choosing best predictor 
x_data_mean_wind=params[[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']]
y_data_mean_wind=params['mean_wind_obs']
x_data_sensitivity=params[[ 'station_z', 'Z_mean_10km']]
# x_data_sensitivity=params[[ 'Z_mean_10km']]
y_data_sensitivity=params['sensitivity']
y_data_linsensitivity=params[['sensitivity']]
x_data_delay=params[[ 'Slope_100m']]
# x_data_delay=params[[ 'glacier_length']]
y_data_delay=params['delay']


# x_data_mean_wind=params[['channel_width_at_station', 'stddev_10', 'T_range_continentality']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['Distance from head','glacier_area']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromhead']]
# y_data_delay=params['delay']
names=list(names)
# for outlier in ['Otemma','Langenferner','Weissee','Corbassiere','Vernatferner','Balcony','Kennikot']:
#     names.remove(outlier)
RMSE_ours=[]
RMSE_era=[]
params['RGI_region']=[s[0:17] for s in params['Glacier_ID']]
same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index


# runfor=same_region_stations
runfor=names
for i,name in enumerate(runfor):
    # if i==1:
    #     continue
    # mean wind prediction
    
    same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index
    same_region_stations=[]
    
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,['Kennikot'])
    model=Lr.fit(X=x_data_mean_wind.drop(index=leave),y=y_data_mean_wind.drop(index=leave))
    x_validate_mean_wind=x_data_mean_wind.loc[[name]]
    if x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]>40:
        x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
    print("Ad-hoc threshold applied")   
    Mean_wind=model.predict(x_validate_mean_wind)


    # sensitivity prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_sensitivity.drop(index=leave))
    x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    S=model.predict(x_validate_sensitivity)

    # linear sensitivity prediction
    # from sklearn.linear_model import LinearRegression
    # Lr=LinearRegression()
    # leave=[name]
    # model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_linsensitivity.drop(index=leave))
    # x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    # S_reg=model.predict(x_validate_sensitivity)

    # delay prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_delay.drop(index=leave),y=y_data_delay.drop(index=leave))
    x_validate_delay=x_data_delay.loc[[name]]
    D=model.predict(x_validate_delay)   


    A=S
    B=-D*S


    D_reg=0
    A_reg=0
    B_reg=0

    # Time series
    name=names[i]
    station_data=pd.read_csv('Universalhourlydatasets//'+name+'.csv')
    station_data['datetime']=pd.to_datetime(station_data['datetime'])
    linear_model=station_data.copy()
    linear_model['temp_model']=linear_model['temp_model']-linear_model['temp_model'].mean()
    linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
    linear_model['ws_model']=linear_model['ws_model']-linear_model['ws_model'].mean()


    lres_wind_speed=np.array(((A*linear_model['temp_model']+B*linear_model['temp_model'].diff())).replace(np.nan,0))+Mean_wind*1
    lreg_wind_speed=station_data['ws'].values*0
    observed_wind_speed=station_data['ws'].values-station_data['ws'].mean()*0
    era5_wind_speed=station_data['ws_model'].values-station_data['ws_model'].mean()*0
    linear_model['ws_pred']=lres_wind_speed
    S_pred=params_stats['S_pred'].loc[name]
    D_pred=params_stats['D_pred'].loc[name]
    U_pred=params_stats['U_pred'].loc[name]
    
    se_S_pred=params_stats['se_S_pred'].loc[name]
    se_D_pred=params_stats['se_D_pred'].loc[name]
    se_U_pred=params_stats['se_U_pred'].loc[name]
    station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
    station_data['datetime'] = pd.to_datetime(station_data['datetime'])
    station_data[station_data['ws']<-0.1]=np.nan
    station_data.dropna(inplace=True)
    station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
    station_data['year'] = station_data['datetime'].dt.year
    year_with_max_data = station_data['year'].value_counts().idxmax()
    station_data.set_index('datetime', inplace=True)
    station_data_hourly = station_data
    station_data_hourly.dropna(inplace=True)
    station_data_hourly = station_data_hourly.resample('1h').mean()
    # station_data_hourly=station_data_hourly.interpolate(method='time')
    station_data = station_data_hourly.reset_index()
    error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
    error_linear_model=error_linear_model[1:]
    linear_model['dt']=linear_model['temp'].diff()
    linear_model=linear_model.dropna()
    linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
    linear_model['ddt']=2*linear_model['dtemp']
    linear_model['d_ws_pred']=np.sqrt((se_U_pred**2)+(S_pred)**2*linear_model['dtemp']+(linear_model['temp']-D_pred*linear_model['dt'])**2*se_S_pred**2+(S_pred**2*D_pred**2*linear_model['dtemp']**2)+(S_pred*linear_model['dt'])**2*se_D_pred**2)
    

    # if not (i==3):
    for l in range(23):
        yodata.append(observed_wind_speed[l])
        ypdata.append(lres_wind_speed[l])
        ypldata.append(lreg_wind_speed[l])
        yedata.append(era5_wind_speed[l])
        yperror.append(linear_model['d_ws_pred'].iloc[l])
        yoerror.append(error_linear_model['ws'].iloc[l])
    RMSE_ours.append(root_mean_squared_error(observed_wind_speed,lres_wind_speed))
    RMSE_era.append(root_mean_squared_error(observed_wind_speed, era5_wind_speed))
    # linear_model['ws']+=
    save_df=pd.DataFrame(columns=['ws_obs','ws_pred','ws_era','temp'])
    save_df['datetime']=np.arange(24)
    save_df['ws_obs']=observed_wind_speed
    save_df['ws_pred']=lres_wind_speed
    save_df['ws_era']=era5_wind_speed
    save_df['temp_era']=station_data['temp_model'].groupby(station_data.datetime.dt.hour).mean()
    # save_df['']
    
    
    save_df.to_csv('Universalhourlydatasets/Predictedhourlydatasets/'+name+'.csv')
jaja=pd.DataFrame()

jaja['u_era5']=yedata#-jaja['u_obs']
jaja['u_S']=ypldata#-jaja['u_obs']
jaja['u_SD']=ypdata
jaja['u_obs']=yodata
jaja['u_error']=yperror
jaja['u_oerror']=yoerror
jaja['glacier_number']=np.arange(23*len(runfor))//23

pd.DataFrame(RMSE_ours).to_csv("RMSE_ours.csv")
pd.DataFrame(RMSE_era).to_csv("RMSE_era5.csv")


# import scienceplots

# plt.style.use(['science','scatter'])

# plt.rcParams["font.sans-serif"] = ["Nimbus Sans L"]
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
# plt.rcParams["axes.titleweight"] = "bold"

fig,ax=fig_row(1)
plt.axes(ax[0])
plt.axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
ax1 = fig.add_axes([0.18,0.55,0.2,0.2])
# plt.axhline(0,color='black',linewidth=1,linestyle='--')
# plt.vlines(jaja['u_obs'],jaja['u_era5'],jaja['u_SD'],linewidth=0.4,linestyle='--',color='gray',zorder=1)
# sns.regplot(data=jaja, x='u_obs',y='their_res',ci=99)
import matplotlib
cmap=matplotlib.colormaps['prism']

# ax[0].text(0.05, 0.95, 'a.', transform=ax[0].transAxes,
#          fontsize=15, fontweight='bold', va='center', ha='center')
# ax[1].text(0.05, 0.95, 'b.', transform=ax[1].transAxes,
#          fontsize=15, fontweight='bold', va='center', ha='center')
plt.axes(ax[0])
import seaborn as sns
# sns.kdeplot(data=jaja, x='u_obs',y='u_era5',ax=plt.gca(),color='dodgerblue',fill=True,levels=20,alpha=0.5,thresh=0.4,linewidth=0.5)
# sns.kdeplot(data=jaja, x='u_obs',y='u_SD',ax=plt.gca(),color='deeppink',fill=True,levels=20,alpha=0.5,thresh=0.3,linewidth=0.5)

# plt.plot(jaja['u_obs'],jaja['u_SD'],label="Model RMSE= "+str(round(np.mean(RMSE_ours),3))+r"$\text{ms}{}^{-1})$",linewidth=0.5,c='deeppink',zorder=2,alpha=0.9)
# plt.plot(jaja['u_obs'],jaja['u_era5'],label="ERA5L RMSE= "+str(round(np.mean(RMSE_era),3))+r"$\text{ms}{}^{-1})$",linewidth=0.5,c='dodgerblue',zorder=1,alpha=0.9)
plt.scatter(jaja['u_obs'],jaja['u_SD'],label="Model",marker='^',edgecolor="black",s= 25,linewidth=0,c='deeppink',zorder=2,alpha=0.6)
ax[0].errorbar(jaja['u_obs'],jaja['u_SD'],xerr=jaja['u_oerror'],yerr=jaja['u_error'],linewidth=0,elinewidth=0.5,color='lightpink',capsize=1,capthick=0.5,alpha=0.5,zorder=-100)
text1=str(round(np.mean(RMSE_ours),2))
plt.scatter(jaja['u_obs'],jaja['u_era5'],label="ERA5L",edgecolor="black",s=25,linewidth=0,c='dodgerblue',zorder=1,alpha=0.6)
text2=str(round(np.mean(RMSE_era),2))

# plt.annotate(text1,(1.8,5),ha='center',color='deeppink')
# plt.annotate(text2,(1.8,4.5),ha='center',color='dodgerblue')

# plt.axvline()
plt.xlabel(r"$u^{obs}\:(\text{ms}{}^{-1})$")
plt.ylabel(r"$u^{pred}\:(\text{ms}{}^{-1})$")
plt.legend(loc='upper left',bbox_to_anchor=(0.0, 0.99),ncols=2)

# plt.xlim(-2,2)    
# plt.ylim(-2,2)
# jaja.dropna(inplace=True)

ax[0].set_facecolor('white')  # Light gray background color
# ax[0].yaxis.set_ticks_position('both')
# ax[0].xaxis.set_ticks_position('both')
# Show the plot
ax[0].set_xlim(0,7)
ax[0].set_ylim(0,7)
# plt.minorticks_on()
ax[0].set_xticks(np.arange(0,7,2))
ax[0].set_xticks(np.arange(1,7,2),minor=True)
ax[0].set_yticks(np.arange(2,7,2))
ax[0].set_yticks(np.arange(1,7,2),minor=True)



df=pd.read_csv("RMSE_ours.csv",index_col=0)
RMSE_ours=df.values
df=pd.read_csv("RMSE_era5.csv",index_col=0)
RMSE_era=df.values
data = []

for rmse in RMSE_era:
    data.append({'Estimator': 'ERA5L', 'RMSE': rmse[0]})

for rmse in RMSE_ours:
    data.append({'Estimator': 'Model', 'RMSE': rmse[0]})

# Create DataFrame
boxplot_df = pd.DataFrame(data)

plt.axes(ax1)

sns.histplot(data=boxplot_df,hue='Estimator',x='RMSE',palette=palette,legend=False,alpha=0.5,fill='False',bins=10)
ax1.tick_params(axis='y', which='major',pad=0)
plt.yticks(rotation=90,size=8)
plt.xticks(rotation=0,size=8)
ax1.set_yticks(np.arange(0,30,10))
ax1.set_yticks(np.arange(5,30,10),minor=True)
ax1.set_xticks(np.arange(0,5,2))
ax1.set_xticks(np.arange(1,5,2),minor=True)

ax1.text(0.5, 0.95, "RMSE", transform=ax1.transAxes,
        fontsize=8, fontweight='regular', va='top', ha='center')
# ax1.text(0.5, 17, text1,
#         fontsize=8, fontweight='regular', va='top', ha='center')
# ax1.text(2.2, 10, text2,
#         fontsize=8, fontweight='regular', va='top', ha='center')

plt.ylabel("")
plt.xlabel("")

plt.savefig("Full_model_prediction.pdf")
plt.show()


Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\527292702.py:199: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\527292702.py:138: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\527292702.py:138: S

Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


In [ ]:
(se_U_pred**2)

In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(RMSE_era))
print("Mean RMSE:", np.round(np.mean(RMSE_era),2))
print("Median RMSE:", np.round(np.median(RMSE_era),2))
print("Inter quartile range:", np.round(np.quantile(RMSE_era,0.25),2),"-",np.round(np.quantile(RMSE_era,0.75),2))
print("Full range:", np.round(np.quantile(RMSE_era,0),2),"-",np.round(np.quantile(RMSE_era,1),2))

In [ ]:
# Clean column output for direct pasting into Word table
for val in RMSE_ours[:, 0]:
    print(round(val,2))


# Supplementary Figure S4: LOOCV

In [12]:

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error,r2_score
import seaborn as sns

 
set_plot_style()

# names=[
# 'arolla',
# 'langenferner',
# 'schiaparelli',
# 'Trakarding',
# 'yala',
# 'lirung',
# 'satopanth',
# 'wm2',
# 'pm3',
# 'pm2',
# 'pm1',
# 'djankuat',
# 'chotashigri',
# 'wm1',
# 'drangdrung',
# 'pyramid',
# 'bishop',
# # 'weisssee',
# 'hef',
# # 'namche',
# # 'balcony',
# 'camp2',
# # 'pm4',
# # 'southcol',
# # 'Denalipass',
# 'kalapathar',
# 'bellavista',
# 'millwitzkeez',
# 'venedegerkees',
# 'vernatferner',
# 'otemma',
# 'corbassiere',
# 'kennicot'
# ]
# Leave one out cross validation


params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)

rmse_lres=[]
rmse_lreg=[]
rmse_era5=[]

yodata=[]
ypdata=[]
ypldata=[]
yedata=[]
yperror=[]
yoerror=[]

params=pd.read_csv(r'New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)

# params.drop(index='Kennikot',inplace=True)
# params.drop(index='Langenferner',inplace=True)
names=params.index

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['rel_distancefromterminus','stddev_1']]

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromterminus']]
# y_data_delay=params['delay']

# After choosing best predictor 
x_data_mean_wind=params[[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']]
y_data_mean_wind=params['mean_wind_obs']
x_data_sensitivity=params[[ 'station_z', 'Z_mean_10km']]
# x_data_sensitivity=params[[ 'Z_mean_10km']]
y_data_sensitivity=params['sensitivity']
y_data_linsensitivity=params[['sensitivity']]
x_data_delay=params[[ 'Slope_100m']]
# x_data_delay=params[[ 'glacier_length']]
y_data_delay=params['delay']


# x_data_mean_wind=params[['channel_width_at_station', 'stddev_10', 'T_range_continentality']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['Distance from head','glacier_area']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromhead']]
# y_data_delay=params['delay']
names=list(names)
# for outlier in ['Otemma','Langenferner','Weissee','Corbassiere','Vernatferner','Balcony','Kennikot']:
#     names.remove(outlier)
RMSE_ours=[]
RMSE_era=[]
params['RGI_region']=[s[0:17] for s in params['Glacier_ID']]
same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index


# runfor=same_region_stations
runfor=names
for i,name in enumerate(runfor):
    # if i==1:
    #     continue
    # mean wind prediction
    
    same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index
    same_region_stations=[name]
    
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,['Kennikot'])
    model=Lr.fit(X=x_data_mean_wind.drop(index=leave),y=y_data_mean_wind.drop(index=leave))
    x_validate_mean_wind=x_data_mean_wind.loc[[name]]
    if x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]>40:
        x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
    print("Ad-hoc threshold applied")   
    Mean_wind=model.predict(x_validate_mean_wind)


    # sensitivity prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_sensitivity.drop(index=leave))
    x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    S=model.predict(x_validate_sensitivity)

    # linear sensitivity prediction
    # from sklearn.linear_model import LinearRegression
    # Lr=LinearRegression()
    # leave=[name]
    # model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_linsensitivity.drop(index=leave))
    # x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    # S_reg=model.predict(x_validate_sensitivity)

    # delay prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_delay.drop(index=leave),y=y_data_delay.drop(index=leave))
    x_validate_delay=x_data_delay.loc[[name]]
    D=model.predict(x_validate_delay)   


    A=S
    B=-D*S


    D_reg=0
    A_reg=0
    B_reg=0

    # Time series
    name=names[i]
    station_data=pd.read_csv('Universalhourlydatasets//'+name+'.csv')
    station_data['datetime']=pd.to_datetime(station_data['datetime'])
    linear_model=station_data.copy()
    linear_model['temp_model']=linear_model['temp_model']-linear_model['temp_model'].mean()
    linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
    linear_model['ws_model']=linear_model['ws_model']-linear_model['ws_model'].mean()


    lres_wind_speed=np.array(((A*linear_model['temp_model']+B*linear_model['temp_model'].diff())).replace(np.nan,0))+Mean_wind*1
    lreg_wind_speed=station_data['ws'].values*0
    observed_wind_speed=station_data['ws'].values-station_data['ws'].mean()*0
    era5_wind_speed=station_data['ws_model'].values-station_data['ws_model'].mean()*0
    linear_model['ws_pred']=lres_wind_speed
    S_pred=params_stats['S_pred'].loc[name]
    D_pred=params_stats['D_pred'].loc[name]
    U_pred=params_stats['U_pred'].loc[name]
    
    se_S_pred=params_stats['se_S_pred'].loc[name]
    se_D_pred=params_stats['se_D_pred'].loc[name]
    se_U_pred=params_stats['se_U_pred'].loc[name]
    station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
    station_data['datetime'] = pd.to_datetime(station_data['datetime'])
    station_data[station_data['ws']<-0.1]=np.nan
    station_data.dropna(inplace=True)
    station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
    station_data['year'] = station_data['datetime'].dt.year
    year_with_max_data = station_data['year'].value_counts().idxmax()
    station_data.set_index('datetime', inplace=True)
    station_data_hourly = station_data
    station_data_hourly.dropna(inplace=True)
    station_data_hourly = station_data_hourly.resample('1h').mean()
    # station_data_hourly=station_data_hourly.interpolate(method='time')
    station_data = station_data_hourly.reset_index()
    error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
    error_linear_model=error_linear_model[1:]
    linear_model['dt']=linear_model['temp'].diff()
    linear_model=linear_model.dropna()
    linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
    linear_model['ddt']=2*linear_model['dtemp']
    linear_model['d_ws_pred']=np.sqrt((se_U_pred**2)+(S_pred)**2*linear_model['dtemp']+(linear_model['temp']-D_pred*linear_model['dt'])**2*se_S_pred**2+(S_pred**2*D_pred**2*linear_model['dtemp']**2)+(S_pred*linear_model['dt'])**2*se_D_pred**2)
    

    # if not (i==3):
    for l in range(23):
        yodata.append(observed_wind_speed[l])
        ypdata.append(lres_wind_speed[l])
        ypldata.append(lreg_wind_speed[l])
        yedata.append(era5_wind_speed[l])
        yperror.append(linear_model['d_ws_pred'].iloc[l])
        yoerror.append(error_linear_model['ws'].iloc[l])
    RMSE_ours.append(root_mean_squared_error(observed_wind_speed,lres_wind_speed))
    RMSE_era.append(root_mean_squared_error(observed_wind_speed, era5_wind_speed))
    # linear_model['ws']+=
    save_df=pd.DataFrame(columns=['ws_obs','ws_pred','ws_era','temp'])
    save_df['datetime']=np.arange(24)
    save_df['ws_obs']=observed_wind_speed
    save_df['ws_pred']=lres_wind_speed
    save_df['ws_era']=era5_wind_speed
    save_df['temp_era']=station_data['temp'].groupby(station_data.datetime.dt.hour).mean()
    # save_df['']
    
    
    save_df.to_csv('Universalhourlydatasets/Predictedhourlydatasets/'+name+'.csv')
jaja=pd.DataFrame()

jaja['u_era5']=yedata#-jaja['u_obs']
jaja['u_S']=ypldata#-jaja['u_obs']
jaja['u_SD']=ypdata
jaja['u_obs']=yodata
jaja['u_error']=yperror
jaja['u_oerror']=yoerror
jaja['glacier_number']=np.arange(23*len(runfor))//23

pd.DataFrame(RMSE_ours).to_csv("RMSE_ours.csv")
pd.DataFrame(RMSE_era).to_csv("RMSE_era5.csv")



# import scienceplots

# plt.style.use(['science','scatter'])

# plt.rcParams["font.sans-serif"] = ["Nimbus Sans L"]
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
# plt.rcParams["axes.titleweight"] = "bold"

fig,ax=fig_row(1)
plt.axes(ax[0])
plt.axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
ax1 = fig.add_axes([0.18,0.55,0.2,0.2])
# plt.axhline(0,color='black',linewidth=1,linestyle='--')
# plt.vlines(jaja['u_obs'],jaja['u_era5'],jaja['u_SD'],linewidth=0.4,linestyle='--',color='gray',zorder=1)
# sns.regplot(data=jaja, x='u_obs',y='their_res',ci=99)
import matplotlib
cmap=matplotlib.colormaps['prism']

# ax[0].text(0.05, 0.95, 'a.', transform=ax[0].transAxes,
#          fontsize=15, fontweight='bold', va='center', ha='center')
# ax[1].text(0.05, 0.95, 'b.', transform=ax[1].transAxes,
#          fontsize=15, fontweight='bold', va='center', ha='center')
plt.axes(ax[0])
import seaborn as sns
# sns.kdeplot(data=jaja, x='u_obs',y='u_era5',ax=plt.gca(),color='dodgerblue',fill=True,levels=20,alpha=0.5,thresh=0.4,linewidth=0.5)
# sns.kdeplot(data=jaja, x='u_obs',y='u_SD',ax=plt.gca(),color='deeppink',fill=True,levels=20,alpha=0.5,thresh=0.3,linewidth=0.5)

# plt.plot(jaja['u_obs'],jaja['u_SD'],label="Model RMSE= "+str(round(np.mean(RMSE_ours),3))+r"$\text{ms}{}^{-1})$",linewidth=0.5,c='deeppink',zorder=2,alpha=0.9)
# plt.plot(jaja['u_obs'],jaja['u_era5'],label="ERA5L RMSE= "+str(round(np.mean(RMSE_era),3))+r"$\text{ms}{}^{-1})$",linewidth=0.5,c='dodgerblue',zorder=1,alpha=0.9)
plt.scatter(jaja['u_obs'],jaja['u_SD'],label="Model",marker='^',edgecolor="black",s= 25,linewidth=0,c='deeppink',zorder=2,alpha=0.6)
# ax[0].errorbar(jaja['u_obs'],jaja['u_SD'],xerr=jaja['u_oerror'],yerr=jaja['u_error'],linewidth=0,elinewidth=0.1,color='deeppink',capsize=1,capthick=0.2,alpha=0.5)
text1=str(round(np.mean(RMSE_ours),2))
plt.scatter(jaja['u_obs'],jaja['u_era5'],label="ERA5L",edgecolor="black",s=25,linewidth=0,c='dodgerblue',zorder=1,alpha=0.6)
text2=str(round(np.mean(RMSE_era),2))

# plt.annotate(text1,(1.8,5),ha='center',color='deeppink')
# plt.annotate(text2,(1.8,4.5),ha='center',color='dodgerblue')

# plt.axvline()
plt.xlabel(r"$u^{obs}\:(\text{ms}{}^{-1})$")
plt.ylabel(r"$u^{pred}\:(\text{ms}{}^{-1})$")
plt.legend(loc='upper left',bbox_to_anchor=(0.0, 0.99),ncols=2)

# plt.xlim(-2,2)    
# plt.ylim(-2,2)
# jaja.dropna(inplace=True)

ax[0].set_facecolor('white')  # Light gray background color
# ax[0].yaxis.set_ticks_position('both')
# ax[0].xaxis.set_ticks_position('both')
# Show the plot
ax[0].set_xlim(0,7)
ax[0].set_ylim(0,7)
# plt.minorticks_on()
ax[0].set_xticks(np.arange(0,7,2))
ax[0].set_xticks(np.arange(1,7,2),minor=True)
ax[0].set_yticks(np.arange(2,7,2))
ax[0].set_yticks(np.arange(1,7,2),minor=True)



df=pd.read_csv("RMSE_ours.csv",index_col=0)
RMSE_ours=df.values
df=pd.read_csv("RMSE_era5.csv",index_col=0)
RMSE_era=df.values
data = []

for rmse in RMSE_era:
    data.append({'Estimator': 'ERA5L', 'RMSE': rmse[0]})

for rmse in RMSE_ours:
    data.append({'Estimator': 'Model', 'RMSE': rmse[0]})

# Create DataFrame
boxplot_df = pd.DataFrame(data)

plt.axes(ax1)

sns.histplot(data=boxplot_df,hue='Estimator',x='RMSE',palette=palette,legend=False,alpha=0.5,fill='False',bins=10)
ax1.tick_params(axis='y', which='major',pad=0)
plt.yticks(rotation=90,size=8)
plt.xticks(rotation=0,size=8)
ax1.set_yticks(np.arange(0,30,10))
ax1.set_yticks(np.arange(5,30,10),minor=True)
ax1.set_xticks(np.arange(0,5,2))
ax1.set_xticks(np.arange(1,5,2),minor=True)

ax1.text(0.5, 0.95, "RMSE", transform=ax1.transAxes,
        fontsize=8, fontweight='regular', va='top', ha='center')
# ax1.text(0.5, 17, text1,
#         fontsize=8, fontweight='regular', va='top', ha='center')
# ax1.text(2.2, 10, text2,
#         fontsize=8, fontweight='regular', va='top', ha='center')

plt.ylabel("")
plt.xlabel("")

plt.savefig("LOOCV.pdf")
plt.show()


Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\225470495.py:199: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\225470495.py:138: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\225470495.py:138: S

Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


In [ ]:
print("Here are all the statistics you need to write the results")
print("=========================================================")
print("Total number of stations:",len(RMSE_era))
print("Mean RMSE:", np.round(np.mean(RMSE_era),2))
print("Median RMSE:", np.round(np.median(RMSE_era),2))
print("Inter quartile range:", np.round(np.quantile(RMSE_era,0.25),2),"-",np.round(np.quantile(RMSE_era,0.75),2))
print("Full range:", np.round(np.quantile(RMSE_era,0),2),"-",np.round(np.quantile(RMSE_era,1),2))

In [ ]:
params_edited=pd.read_csv("params_edited.csv")

In [ ]:
for val in RMSE_era[:, 0]:
    print(round(val,2))

# Time scales dependence

In [13]:
from sklearn.metrics import root_mean_squared_error
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load parameters and preprocess
params = pd.read_csv(r'New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)
x_data_mean_wind=params[[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']]
y_data_mean_wind=params['mean_wind_obs']
x_data_sensitivity=params[[ 'station_z', 'Z_mean_10km']]
# x_data_sensitivity=params[[ 'Z_mean_10km']]
y_data_sensitivity=params['sensitivity']
y_data_linsensitivity=params[['sensitivity']]
x_data_delay=params[[ 'Slope_100m']]
# x_data_delay=params[[ 'glacier_length']]
y_data_delay=params['delay']


# Station names
names=params.index
# Open PDF file
pdf_path = "subhourly_monthly.pdf"

#RMSE_9H
pdf_path = "subhourly_monthly.pdf"
RMSE_df=pd.DataFrame(columns=['Estimator','RMSE',"RMSE_era"])
plt.ioff()
with PdfPages(pdf_path) as pdf:
    for i, name in enumerate(names):
        leave=['Kennikot']

        # Mean wind prediction
        model = LinearRegression().fit(X=x_data_mean_wind.drop(index=leave), y=y_data_mean_wind.drop(index=leave))
        x_validate_mean_wind = x_data_mean_wind.loc[[name]]
        if x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]>40:
            x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
            print("Ad-hoc threshold applied")           
        Mean_wind = model.predict(x_validate_mean_wind)
        leave=['Langenferner']
        # Sensitivity prediction
        model = LinearRegression().fit(X=x_data_sensitivity.drop(index=leave), y=y_data_sensitivity.drop(index=leave))
        x_validate_sensitivity = x_data_sensitivity.loc[[name]]
        S = model.predict(x_validate_sensitivity)

        # Linear sensitivity prediction
        leave=['Langenferner']
        model = LinearRegression().fit(X=x_data_sensitivity.drop(index=leave), y=y_data_linsensitivity.drop(index=leave))
        x_validate_sensitivity = x_data_sensitivity.loc[[name]]
        S_reg = model.predict(x_validate_sensitivity)

        # Delay prediction
        leave=['Langenferner']
        model = LinearRegression().fit(X=x_data_delay.drop(index=leave), y=y_data_delay.drop(index=leave))
        x_validate_delay = x_data_delay.loc[[name]]
        D = model.predict(x_validate_delay)

        A = S
        B = -D * S

        D_reg = 0
        A_reg = S_reg
        B_reg = 0

        # RMSE_seasonal=[]
        t_list=[1,6,24,48,24*3,24*4,24*5,24*6,24*7,24*8,24*9,24*10]
        for j,t in enumerate(t_list):
            t_str=str(t)+"h"
            # Time series data
            station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
            station_data['datetime'] = pd.to_datetime(station_data['datetime'])
            station_data[station_data['ws']<-0.1]=np.nan
            station_data.dropna(inplace=True)
            station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
            station_data['year'] = station_data['datetime'].dt.year
            year_with_max_data = station_data['year'].value_counts().idxmax()
            station_data=station_data[station_data['year']==year_with_max_data]
            station_data.set_index('datetime', inplace=True)
            station_data_hourly = station_data.rolling(t,center=True).mean()
            station_data_hourly.dropna(inplace=True)
            station_data_hourly = station_data_hourly.resample('1h').mean()
            # station_data_hourly=station_data_hourly.interpolate(method='time')
            station_data = station_data_hourly.reset_index()

            linear_model = station_data.copy()
            linear_model['temp'] = linear_model['temp'] - linear_model['temp'].mean()
            mean_wind_cache= linear_model['ws'].mean()
            linear_model['ws'] = linear_model['ws'] - linear_model['ws'].mean()
            linear_model['dt']=linear_model['temp'].diff().replace(np.nan, 0)
            lres_wind_speed = np.array(((A * linear_model['temp'] + B *linear_model['temp'].diff())).replace(np.nan, 0)) + Mean_wind
            linear_model['ws_pred'] = lres_wind_speed
            linear_model['ws']=linear_model['ws']+mean_wind_cache
            params_stats=pd.read_csv("params_stats.csv")
            params_stats.set_index('glacier_name',inplace=True)
            
            S_pred=params_stats['S_pred'].loc[name]
            D_pred=params_stats['D_pred'].loc[name]
            U_pred=params_stats['U_pred'].loc[name]
            
            se_S_pred=params_stats['se_S_pred'].loc[name]
            se_D_pred=params_stats['se_D_pred'].loc[name]
            se_U_pred=params_stats['se_U_pred'].loc[name]
            
            linear_model['d_ws_pred']=np.sqrt(((linear_model['temp']-D_pred*linear_model['dt'])**2)*se_S_pred**2+((S_pred*linear_model['dt'])**2)*se_D_pred**2+se_U_pred**2) # u fit error is zero
            linear_model['ws_pred_upper']=linear_model['ws_pred']+linear_model['d_ws_pred']
            linear_model['ws_pred_lower']=linear_model['ws_pred']-linear_model['d_ws_pred']

            linear_model['ws']=linear_model['ws']*(linear_model['ws']>0)
            linear_model['ws_pred']=linear_model['ws_pred']*(linear_model['ws_pred']>0)

            
            
            plt.figure(figsize=(10,5))

            linear_model['ws_pred'].rolling(1).mean().plot(linewidth=0.5)
            linear_model['ws'].rolling(1).mean().plot(linewidth=0.5)
            ax=plt.gca()
            ax.fill_between(linear_model['ws_pred'].rolling(1).mean().index,linear_model['ws_pred_upper'].rolling(1).mean(),linear_model['ws_pred_lower'].rolling(1).mean(),alpha=0.5,color='gray')
            plt.title(name)
            # plt.xlim(linear_model['ws_pred'].rolling(1).mean().quantile(0.01),linear_model['ws_pred'].rolling(1).mean().quantile(0.99))
            # plt.ylim(linear_model['ws'].rolling(1).mean().quantile(0.01),linear_model['ws'].rolling(1).mean().quantile(0.99))
            plt.ylabel("Wind speed (m/s)")
            # plt.axline((0, 0), (1, 1), color='deeppink', zorder=100)
            plt.grid(True)
            pdf.savefig()
            plt.close()
            
            linear_model.dropna(inplace=True)
            RMSE=root_mean_squared_error(linear_model['ws'],linear_model['ws_pred'])
            RMSE_era=root_mean_squared_error(linear_model['ws'],linear_model['ws_model'])
            print("Station",name, "RMSE",RMSE,RMSE_era)
            RMSE_df.loc[len(RMSE_df)] = {'Estimator': t_str, 'RMSE': RMSE,'RMSE_era':RMSE_era}
            # Save each plot to a new page
            
            
            # print(RMSE_df)
            # print("Mean wind:",mean_wind_cache)
            # Mean only

Station Arolla RMSE 1.7349826275041533 2.894914632637717
Station Arolla RMSE 1.4867041387799194 2.7625060814784823
Station Arolla RMSE 1.099044228509327 2.615192942919971
Station Arolla RMSE 0.9162517890002706 2.5641067657386527
Station Arolla RMSE 0.8260899722686453 2.538900153887502
Station Arolla RMSE 0.7693610242823695 2.525268127332169
Station Arolla RMSE 0.7220097722599931 2.5128457223768996
Station Arolla RMSE 0.6891431073005625 2.5058677403706873
Station Arolla RMSE 0.6649352633782376 2.503002613657828
Station Arolla RMSE 0.6424798617232961 2.499852785081694
Station Arolla RMSE 0.6229367896890111 2.4991768241663523
Station Arolla RMSE 0.6065053035361899 2.501565870006976
Station Balcony RMSE 2.6992437872395127 3.6784552663326364
Station Balcony RMSE 2.3587768170009547 3.4353714970373073
Station Balcony RMSE 1.890157352925045 3.1430222903623313
Station Balcony RMSE 1.653032821710185 3.0059212811443388
Station Balcony RMSE 1.5094927081007026 2.932804960242751
Station Balcony RMSE

C:\Users\krish\AppData\Local\Temp\ipykernel_14908\4267402189.py:73: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


Station Chottashigri RMSE 2.202601973174441 2.5306451136770667


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\4267402189.py:73: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


Station Chottashigri RMSE 1.9552535331899565 2.3302214693840777


...

Station Chottashigri RMSE 1.157976834534102 1.8606512962515729


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\4267402189.py:73: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


Station Chottashigri RMSE 1.0979525132558154 1.8390667602330628
Station Djankuat RMSE 5.1646632957996275 8.081584164894881
Station Djankuat RMSE 4.898028581610053 7.897364739402677
Station Djankuat RMSE 4.231756406469337 7.419506709732904
Station Djankuat RMSE 4.200181333707166 7.391633501546305
Station Djankuat RMSE 4.199782025121356 7.390551053195099
Station Djankuat RMSE 4.208224887731293 7.398416437146535
Station Djankuat RMSE 4.220346007155502 7.410932899456599
Station Djankuat RMSE 4.239389021219766 7.430289388645847
Station Djankuat RMSE 4.260297863330931 7.4509276408254586
Station Djankuat RMSE 4.279479705922684 7.4696650639991296
Station Djankuat RMSE 4.2968748562129075 7.487068850649558
Station Djankuat RMSE 4.313260303351733 7.504078933227101
Station Drangdrung RMSE 1.5301606123828453 3.7856343332926135
Station Drangdrung RMSE 1.2637680149508919 3.712001827376911
Station Drangdrung RMSE 0.9291619211942305 3.6313862479727734
Station Drangdrung RMSE 0.8544883869670392 3.613930

C:\Users\krish\AppData\Local\Temp\ipykernel_14908\4267402189.py:39: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\4267402189.py:39: S

Station Kennikot RMSE 1.2745549243462984 1.4139835831511516
Station Kennikot RMSE 1.2221803528918047 1.3271454155271736
Station Kennikot RMSE 1.220839019277806 1.1540191984300845
Station Kennikot RMSE 1.2207133161658836 1.1471552631367787
Station Kennikot RMSE 1.224512680145968 1.1457709554166362
Station Kennikot RMSE 1.225105441446419 1.1469087952060497
Station Kennikot RMSE 1.22049864935408 1.1486634159082367
Station Kennikot RMSE 1.2165318455041279 1.1508146214787818
Station Kennikot RMSE 1.2143070082719671 1.1522854238519238
Station Kennikot RMSE 1.2138189403092796 1.1539212129834735
Station Kennikot RMSE 1.2140641560193197 1.1555571064797572
Station Kennikot RMSE 1.214604380913992 1.1570969125961286
Station Langenferner RMSE 2.28867666193434 3.1794996343815476
Station Langenferner RMSE 2.082242838701993 3.042834513519526
Station Langenferner RMSE 1.7811295997242813 2.8928047400273287
Station Langenferner RMSE 1.6488786140320542 2.833988170184827
Station Langenferner RMSE 1.5624942

In [ ]:
# RMSE_df.to_csv("RMSE_timescalesxyz.csv")


In [14]:
RMSE_df=pd.read_csv("RMSE_timescales.csv")
# Convert from wide to long format
RMSE_df_long = pd.melt(
    RMSE_df,
    id_vars=['Estimator'],
    value_vars=['RMSE', 'RMSE_era'],
    var_name='Model',
    value_name='RMSE_Value'
)

# Rename models for clarity (optional)
RMSE_df_long['Model'] = RMSE_df_long['Model'].replace({
    'RMSE': 'Model',
    'RMSE_era': 'ERA5L'
})
RMSE_df_long

,Estimator,Model,RMSE_Value
0,1h,Model,1.734983
1,6h,Model,1.486704
2,24h,Model,1.099044
3,48h,Model,0.916252
4,72h,Model,0.826090
...,...,...,...
667,144h,ERA5L,0.315750
668,168h,ERA5L,0.310020
669,192h,ERA5L,0.304426
670,216h,ERA5L,0.299493


In [15]:
fig,ax=fig_row(1,figsize=(5,4))
RMSE_df['RMSE'] = RMSE_df['RMSE'].clip(lower=0)
sns.boxplot(data=RMSE_df,x='Estimator',y='RMSE',color='lightskyblue')
# sns.boxplot(data=RMSE_df,x='Estimator',y='RMSE',palette='coolwarm_r',robust=True,saturation=0.9,density_norm='count')
plt.xlabel("Moving average window")
plt.ylabel(r"RMSE (ms${}^{-1}$)")
plt.xticks(ticks=plt.gca().get_xticks(),labels=['1h','6h','1d','2d','3d','4d','5d','6d','7d','8d','9d','10d'])
plt.ylim(0,7)
plt.savefig("Timescales.pdf")
plt.show()

# Figure 8 : Minimum stations and timescales

In [17]:
beta=np.load("beta.npy")
fig,ax=plt.subplots(nrows=2,figsize=(7,10))
plt.subplots_adjust(hspace=0.3)
plt.axes(ax[0])
ax[0].text(0.03, 0.925, 'a.', transform=ax[0].transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
ax[1].text(0.03, 0.925, 'b.', transform=ax[1].transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
sns.boxplot(np.transpose(beta[::-1,9,:]),color='lightblue',ax=ax[0],linecolor='black')
sns.scatterplot(x=np.arange(16),y=np.mean(beta[::-1,9,:],axis=1),color='black',s=15,zorder=20,ax=ax[0])

# plt.plot(beta[:,9,:])
plt.xticks(ticks=np.arange(1,16,2),labels=29-np.arange(1,16,2)[::-1])
plt.xticks(ticks=np.arange(0,16,2),minor=True)
plt.xlim(0.5,16)
plt.ylim(top=1.15)
plt.tick_params(axis='y',rotation=90)

plt.yticks(np.arange(0.7,1.1,0.1))
plt.yticks(np.arange(0.75,1.1,0.1),minor=True)
plt.xlabel("N")
plt.ylabel(r"RMSE "+r"(ms$^{-1}$)")
plt.legend()
plt.show()
# plt.savefig(r"Applicability.pdf", bbox_inches='tight')

plt.axes(ax[1])
plt.yticks(np.arange(0,6.1,1))
plt.yticks(np.arange(0.5,6.1,1),minor=True)
RMSE_df=pd.read_csv("RMSE_timescales.csv")
RMSE_df['RMSE'] = RMSE_df['RMSE'].clip(lower=0)
RMSE_df['Estimator_n']=[int(n[:-1]) for n in RMSE_df['Estimator']]
RMSE_df.sort_values('Estimator_n',inplace=True)
sns.boxplot(data=RMSE_df,x='Estimator_n',y='RMSE',color='lightblue',ax=ax[1],linecolor='black')
sns.scatterplot(x=plt.gca().get_xticks(),y=RMSE_df['RMSE'].groupby(RMSE_df['Estimator_n']).mean(),color='black',s=15,zorder=100,ax=ax[1])
# sns.boxplot(data=RMSE_df,x='Estimator',y='RMSE',palette='coolwarm_r',robust=True,saturation=0.9,density_norm='count')
plt.xlabel("Moving average window (hr)")
plt.ylabel(r"RMSE (ms${}^{-1}$)")
mask=np.array([True,True,True,True,True,True,True,True,True,True,True,True])
ticklocs=np.array(plt.gca().get_xticks())
# plt.xticks(ticks=ticklocs[mask],
#                                  labels=np.array(['1h','6h','1d','2d','3d','4d','5d','6d','7d','8d','9d','10d'])[mask])
# plt.xticks(ticks=ticklocs[~mask],
#                                  minor=True)
plt.yticks(rotation=90)
plt.ylim(0,6)
plt.savefig("Figure8.pdf")
plt.show()


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\2036777469.py:23: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [ ]:
# set_plot_style()
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
sns.boxplot(data=RMSE_df_long, x='Estimator', y='RMSE_Value', hue='Model', palette='coolwarm_r')
plt.xlabel("Moving average window")
plt.ylabel(r"RMSE (ms${}^{-1}$)")
plt.xticks(ticks=plt.gca().get_xticks(),labels=['1h','6h','1d','2d','3d','4d','5d','6d','7d','8d','9d','10d'])
plt.ylim(0, 7)
plt.legend()
plt.tight_layout()
plt.savefig("Timescales.pdf")
plt.show()


# Time scales_fit

In [ ]:
from sklearn.metrics import root_mean_squared_error
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load parameters and preprocess
params = pd.read_csv(r'New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)
x_data_mean_wind=params[[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']]
y_data_mean_wind=params['mean_wind_obs']
x_data_sensitivity=params[[ 'station_z', 'Z_mean_10km']]
# x_data_sensitivity=params[[ 'Z_mean_10km']]
y_data_sensitivity=params['sensitivity']
y_data_linsensitivity=params[['sensitivity']]
x_data_delay=params[[ 'Slope_100m']]
# x_data_delay=params[[ 'glacier_length']]
y_data_delay=params['delay']


# Station names
names=params.index
# Open PDF file
pdf_path = "subhourly_monthly.pdf"

#RMSE_9H
pdf_path = "subhourly_monthly.pdf"
RMSE_df=pd.DataFrame(columns=['Estimator','RMSE','RMSE_era'])
plt.ioff()
with PdfPages(pdf_path) as pdf:
    for i, name in enumerate(names):
        leave=['Kennikot']

        # Mean wind prediction
        model = LinearRegression().fit(X=x_data_mean_wind.drop(index=leave), y=y_data_mean_wind.drop(index=leave))
        x_validate_mean_wind = x_data_mean_wind.loc[[name]]
        if x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]>40:
            x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
            print("Ad-hoc threshold applied")           
        Mean_wind = params['mean_wind_obs'].loc[name]
        leave=['Langenferner']
        # Sensitivity prediction
        model = LinearRegression().fit(X=x_data_sensitivity.drop(index=leave), y=y_data_sensitivity.drop(index=leave))
        x_validate_sensitivity = x_data_sensitivity.loc[[name]]
        S = params['sensitivity'].loc[name]

        # Linear sensitivity prediction
        leave=['Langenferner']
        model = LinearRegression().fit(X=x_data_sensitivity.drop(index=leave), y=y_data_linsensitivity.drop(index=leave))
        x_validate_sensitivity = x_data_sensitivity.loc[[name]]
        S_reg = model.predict(x_validate_sensitivity)

        # Delay prediction
        leave=['Langenferner']
        model = LinearRegression().fit(X=x_data_delay.drop(index=leave), y=y_data_delay.drop(index=leave))
        x_validate_delay = x_data_delay.loc[[name]]
        D = params['delay'].loc[name]

        A = S
        B = -D * S

        D_reg = 0
        A_reg = S_reg
        B_reg = 0

        # RMSE_seasonal=[]
        t_list=[1,6,24,48,24*3,24*4,24*5,24*6,24*7,24*8,24*9,24*10]
        for j,t in enumerate(t_list):
            t_str=str(t)+"h"
            # Time series data
            station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
            station_data['datetime'] = pd.to_datetime(station_data['datetime'])
            station_data[station_data['ws']<-0.1]=np.nan
            station_data.dropna(inplace=True)
            station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
            station_data['year'] = station_data['datetime'].dt.year
            year_with_max_data = station_data['year'].value_counts().idxmax()
            station_data=station_data[station_data['year']==year_with_max_data]
            station_data.set_index('datetime', inplace=True)
            station_data_hourly = station_data.rolling(t,center=True).mean()
            station_data_hourly.dropna(inplace=True)
            station_data_hourly = station_data_hourly.resample('1h').mean()
            # station_data_hourly=station_data_hourly.interpolate(method='time')
            station_data = station_data_hourly.reset_index()

            linear_model = station_data.copy()
            linear_model['temp'] = linear_model['temp'] - linear_model['temp'].mean()
            mean_wind_cache= linear_model['ws'].mean()
            linear_model['ws'] = linear_model['ws'] - linear_model['ws'].mean()
            linear_model['dt']=linear_model['temp'].diff().replace(np.nan, 0)
            lres_wind_speed = np.array(((A * linear_model['temp'] + B *linear_model['temp'].diff())).replace(np.nan, 0)) + Mean_wind
            linear_model['ws_pred'] = lres_wind_speed
            linear_model['ws']=linear_model['ws']+mean_wind_cache
            params_stats=pd.read_csv("params_stats.csv")
            params_stats.set_index('glacier_name',inplace=True)
            
            S_pred=params_stats['S_pred'].loc[name]
            D_pred=params_stats['D_pred'].loc[name]
            U_pred=params_stats['U_pred'].loc[name]
            
            se_S_pred=params_stats['se_S_pred'].loc[name]
            se_D_pred=params_stats['se_D_pred'].loc[name]
            se_U_pred=params_stats['se_U_pred'].loc[name]
            
            linear_model['d_ws_pred']=np.sqrt(((linear_model['temp']-D_pred*linear_model['dt'])**2)*se_S_pred**2+((S_pred*linear_model['dt'])**2)*se_D_pred**2+se_U_pred**2) # u fit error is zero
            linear_model['ws_pred_upper']=linear_model['ws_pred']+linear_model['d_ws_pred']
            linear_model['ws_pred_lower']=linear_model['ws_pred']-linear_model['d_ws_pred']

            linear_model['ws']=linear_model['ws']*(linear_model['ws']>0)
            linear_model['ws_pred']=linear_model['ws_pred']*(linear_model['ws_pred']>0)

            
            
            plt.figure(figsize=(10,5))

            linear_model['ws_pred'].rolling(1).mean().plot(linewidth=0.5)
            linear_model['ws'].rolling(1).mean().plot(linewidth=0.5)
            ax=plt.gca()
            ax.fill_between(linear_model['ws_pred'].rolling(1).mean().index,linear_model['ws_pred_upper'].rolling(1).mean(),linear_model['ws_pred_lower'].rolling(1).mean(),alpha=0.5,color='gray')
            plt.title(name)
            # plt.xlim(linear_model['ws_pred'].rolling(1).mean().quantile(0.01),linear_model['ws_pred'].rolling(1).mean().quantile(0.99))
            # plt.ylim(linear_model['ws'].rolling(1).mean().quantile(0.01),linear_model['ws'].rolling(1).mean().quantile(0.99))
            plt.ylabel("Wind speed (m/s)")
            # plt.axline((0, 0), (1, 1), color='deeppink', zorder=100)
            plt.grid(True)
            pdf.savefig()
            plt.close()
            
            linear_model.dropna(inplace=True)
            RMSE=root_mean_squared_error(linear_model['ws'],linear_model['ws_pred'])
            RMSE_era=root_mean_squared_error(linear_model['ws_model'],linear_model['ws'])
            print("Station",name, "RMSE",RMSE,RMSE_era)
            RMSE_df.loc[len(RMSE_df)] = {'Estimator': t_str, 'RMSE': RMSE,'RMSE_era': RMSE_era}
            # Save each plot to a new page
            
            
            # print(RMSE_df)
            # print("Mean wind:",mean_wind_cache)
            # Mean only

In [ ]:
RMSE_df.groupby('Estimator').mean()

# Implications on melt estimate

In [18]:
# Create era5_turbulence.csv
Turbulence_table=pd.DataFrame(columns=['upred'	,'uera5'	,'uobs','Tdewp'	,'e'	,'es',	'P'	,'Tera'	,'Qshf(obs)',	'Qlhf(obs)',	'Qthf(obs)'	,'Qshf(ERA5L)'	,'Qlhf(ERA5L)',	'Qthf(ERA5L)'	,'Qshf(param)'	,'Qlhf(param)',	'Qthf(param)'],index=params.index)
columns=['Tdewp','P']
u_pred_df=pd.read_csv("Mean_wind_prediction.csv")
u_pred_df.set_index('glacier_name',inplace=True)
Turbulence_table['upred']=u_pred_df['upred']
Turbulence_table['uobs']=params['mean_wind_obs']
Turbulence_table['uera5']=params['mean_wind_era5']
Turbulence_table['Tera']=params['mean_temp_era5']

for name in params.index:
    data=pd.read_csv("Universalhourlydatasets/"+name+".csv")
    Turbulence_table['Tdewp'].loc[name]=data['dewpoint_temp'].mean()-273.15
    Turbulence_table['P'].loc[name]=data['pressure'].mean()
    
# Turbulence_table['upred']=Turbulence_table['uobs']/Turbulence_table['uobs']*Turbulence_table['uobs'].mean()
# Turbulence_table['uera5']=Turbulence_table['uera5']/Turbulence_table['uera5']*Turbulence_table['uera5'].mean()
# Turbulence_table['Tera']=Turbulence_table['uera5']/Turbulence_table['uera5']*Turbulence_table['Tera'].mean()
# Turbulence_table['Tdewp']=Turbulence_table['uera5']/Turbulence_table['uera5']*Turbulence_table['Tdewp'].mean()
# Turbulence_table['es']=[100*6.1121*np.exp((18.678-t/234.5)*(t/(257.14+t))) if t>0 else 100*6.1115*np.exp((23.036-t/333.7)*(t/(279.82+t))) for t in Turbulence_table['Tera']]
Turbulence_table['es']=[100*6.1121*np.exp(17.67*t/(t+243.5)) for t in Turbulence_table['Tera']]
Turbulence_table['e']=[100*6.1121*np.exp(17.67*td/(td+243.5)) for td in Turbulence_table['Tdewp']]
Turbulence_table['Qshf(obs)']=0.0129*0.002*Turbulence_table['uobs']*Turbulence_table['P']*Turbulence_table['Tera']
Turbulence_table['Qshf(ERA5L)']=0.0129*0.002*Turbulence_table['uera5']*Turbulence_table['P']*Turbulence_table['Tera']
Turbulence_table['Qshf(param)']=0.0129*0.002*Turbulence_table['upred']*Turbulence_table['P']*Turbulence_table['Tera']
Turbulence_table['Qlhf(obs)']=0.622*1.29*22.6*10**5*0.002*Turbulence_table['uobs']*(Turbulence_table['e']-Turbulence_table['es'])/(1.013*10**5)
Turbulence_table['Qlhf(ERA5L)']=0.622*1.29*22.6*10**5*0.002*Turbulence_table['uera5']*(Turbulence_table['e']-Turbulence_table['es'])/(1.013*10**5)
Turbulence_table['Qlhf(param)']=0.622*1.29*22.6*10**5*0.002*Turbulence_table['upred']*(Turbulence_table['e']-Turbulence_table['es'])/(1.013*10**5)
Turbulence_table['Qthf(obs)']=Turbulence_table['Qshf(obs)']+Turbulence_table['Qlhf(obs)']
Turbulence_table['Qthf(ERA5L)']=Turbulence_table['Qshf(ERA5L)']+Turbulence_table['Qlhf(ERA5L)']
Turbulence_table['Qthf(param)']=Turbulence_table['Qshf(param)']+Turbulence_table['Qlhf(param)']
Turbulence_table.to_csv('era5_turbulence.csv')

C:\Users\krish\AppData\Local\Temp\ipykernel_14908\256024568.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Turbulence_table['Tdewp'].loc[name]=data['dewpoint_temp'].mean()-273.15
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\256024

In [360]:
Turbulence_table['upred'].mean()

2.670124961642742

# Fig. 9 TURBULENCE plot

In [19]:
# %matplotlib qt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error

fig,ax=fig_row(1)
ax1 = fig.add_axes([0.18,0.55,0.2,0.2])

plt.axes(ax[0])

# plt.style.use('default')

# plt.axhline(0,color='black',linewidth=1,linestyle='--')
plt.axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
# plt.vlines(jaja['Qthf(obs)'],jaja['Qthf(param)'],jaja['Qthf(ERA5L)'],linewidth=0.4,linestyle='--',color='gray',zorder=1)
import matplotlib
import seaborn as sns
# sns.regplot(data=jaja, x='Qthf(obs)',y='Qthf(ERA5L)',ci=99)
import matplotlib

jaja=pd.read_csv(r"era5_turbulence.csv")

t_rmse_ours=abs(jaja['Qthf(obs)']-jaja['Qthf(param)'])
t_rmse_era5=abs(jaja['Qthf(obs)']-jaja['Qthf(ERA5L)'])

plt.scatter(jaja['Qthf(obs)'],jaja['Qthf(param)'],label="Model",edgecolor='black',s=50,c='deeppink',marker='^',zorder=3,cmap='Paired',linewidth=0,alpha=0.6)

text1="RMSE = "+str(round(np.mean(t_rmse_ours),1))+r" $\text{Wm}{}^{-2}$"
plt.scatter(jaja['Qthf(obs)'],jaja['Qthf(ERA5L)'],label="ERA5L",edgecolor="black",s=50,c='dodgerblue',marker='o',zorder=2,linewidth=0,alpha=0.6)
text2="RMSE = "+str(round(np.mean(t_rmse_era5),1))+r" $\text{Wm}{}^{-2}$"
# plt.annotate(text1,(-20,22),ha='center',color='deeppink')
# plt.annotate(text2,(-20,15),ha='center',color='dodgerblue')

# sns.kdeplot(data=jaja, x='Qthf(obs)',y='Qthf(param)',ax=plt.gca(),color='deeppink',fill=True,levels=20,alpha=0.5,thresh=0.4,linewidth=2)
# sns.kdeplot(data=jaja, x='Qthf(obs)',y='Qthf(ERA5L)',ax=plt.gca(),color='dodgerblue',fill=True,levels=20,alpha=0.5,thresh=0.4,linewidth=2)

# plt.axvline()
plt.xlabel(r"$Q^{obs}_{thf}\:(\text{Wm}{}^{-2})$")
plt.ylabel(r"$Q^{pred}_{thf}\:(\text{Wm}{}^{-2})$")
plt.yticks(rotation=90)
# plt.minorticks_on()
# plt.rcParams['font.size']=15
plt.legend(loc='upper left',bbox_to_anchor=(0.02,0.95),ncols=2)

# plt.xlim(-2,2)
# plt.ylim(-2,2)
# jaja.dropna(inplace=True)




# ax.yaxis.tick_right()
# ax[0].yaxis.set_ticks_position('both')
# ax[0].xaxis.set_ticks_position('both')
# Show the plot
# plt.show()
plt.xlim(-40,55)
plt.ylim(-40,55)
data=[]
for rmse in t_rmse_era5:
    data.append({'Estimator': 'ERA5L', 'RMSE': rmse})

for rmse in t_rmse_ours:
    data.append({'Estimator': 'Model', 'RMSE': rmse})

# Create DataFrame
boxplot_df = pd.DataFrame(data)

plt.axes(ax1)

sns.histplot(data=boxplot_df,hue='Estimator',x='RMSE',palette=palette,legend=False,alpha=0.5,fill='False',bins=10)
ax1.tick_params(axis='y', which='major',pad=0)
plt.yticks(rotation=90,size=8)
plt.xticks(rotation=0,size=8)
ax1.set_yticks(np.arange(0,30,10))
ax1.set_yticks(np.arange(5,30,10),minor=True)
ax1.set_xticks(np.arange(0,35,10))
ax1.set_xticks(np.arange(5,35,10),minor=True)

ax1.text(0.5, 0.95, "RMSE", transform=ax1.transAxes,
        fontsize=8, fontweight='regular', va='top', ha='center')
# ax1.text(0.5, 17, text1,
#         fontsize=8, fontweight='regular', va='top', ha='center')
# ax1.text(2.2, 10, text2,
#         fontsize=8, fontweight='regular', va='top', ha='center')

plt.ylabel("")
plt.xlabel("")



plt.savefig("Turbulence_plot.pdf",bbox_inches='tight')
# jaja.groupby('glacier_number').mean().to_csv("ENERGY_BALANCE.csv")
# plt.close()


# %matplotlib qt

plt.show()

C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3067893131.py:28: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  plt.scatter(jaja['Qthf(obs)'],jaja['Qthf(param)'],label="Model",edgecolor='black',s=50,c='deeppink',marker='^',zorder=3,cmap='Paired',linewidth=0,alpha=0.6)


In [333]:
print("ERA5 Mean over all stations:",jaja['Qthf(ERA5L)'].mean())
print("Param Mean over all stations:",jaja['Qthf(param)'].mean())
print("Actual Mean over all stations:",jaja['Qthf(obs)'].mean())
print("ERA5 range over all stations:",jaja['Qthf(ERA5L)'].max()-jaja['Qthf(ERA5L)'].min())
print("Param range over all stations:",jaja['Qthf(param)'].max()-jaja['Qthf(param)'].min())
print("Actual range over all stations:",jaja['Qthf(obs)'].max()-jaja['Qthf(obs)'].min())


ERA5 Mean over all stations: 3.397178463645999
Param Mean over all stations: 6.887599777717525
Actual Mean over all stations: 7.1957525020493165
ERA5 range over all stations: 58.10650859801296
Param range over all stations: 70.47671863348417
Actual range over all stations: 67.22190699351486


In [345]:
jaja['rmse(ERA5L)']=np.abs(jaja['Qthf(ERA5L)']-jaja['Qthf(obs)'])
jaja['rmse(param)']=np.abs(jaja['Qthf(param)']-jaja['Qthf(obs)'])

print("ERA5 Mean rmse over all stations:",jaja['rmse(ERA5L)'].mean())
print("Param Mean rmse over all stations:",jaja['rmse(param)'].mean())
print("Ratio:",jaja['rmse(ERA5L)'].mean()/jaja['rmse(param)'].mean())
print("Difference:",jaja['rmse(ERA5L)'].mean()-jaja['rmse(param)'].mean())

ERA5 Mean rmse over all stations: 12.212496339605929
Param Mean rmse over all stations: 2.6611229217492745
Ratio: 4.589226690655129
Difference: 9.551373417856654


In [50]:
positive_stations=jaja[jaja['QH(obs)']>0]
print((positive_stations['QH(param)']/positive_stations['QH(ERA5L)']).mean())
negative_stations=jaja[jaja['QH(obs)']<0]
print((negative_stations['QH(param)']/negative_stations['QH(ERA5L)']).mean())

print(len(positive_stations),len(negative_stations))

3.454031253658377
3.1848043400134145
17 11


In [7]:
numeric_columns=params.select_dtypes(include="number").columns.values
numeric_columns = [col for col in numeric_columns if not col.lower().startswith("unnamed") and not np.isin(col,['rel_heightfromglacier','Heightfromglacier','CW2','mean_wind_era5','mean_wind_obs','mean_temp_obs','rel_distance_from_terminus','Distance from terminus',"mean_along_valley%",'score','lat','lon','A','B','sensitivity','delay','Down_glacier_direction','Z_max','Z_min','mean_wind_era','localtime',
                                                                                                                	'channel_width_100m_from_lowest_point','rel_heightfromglacier',	'score'	,'mean_along_valley%',	'data_length_days',	'mean_abs_along_valley%','Mean_elev'])]
pd.DataFrame(numeric_columns)

,0
0,left_right
1,CW
2,relief_1km
3,relief_5km
4,relief_10km
5,stddev_1km
6,stddev_5km
7,stddev_10km
8,Slope_100m
9,Slope_1km


# Applicability

In [20]:

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error,r2_score
import seaborn as sns

 

beta=np.zeros((16,10,100))
for iteration in range(100):
    for leave_n in [15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]:
        rmse_lres=[]
        rmse_lreg=[]
        rmse_era5=[]

        yodata=[]
        ypdata=[]
        ypldata=[]
        yedata=[]

        params=pd.read_csv(r'New_bigtable/Universalparams.csv')
        params.set_index('glacier_name',inplace=True)
        import random
        from random import seed
        # seed(10)
        leave_stations=random.sample(list(params.drop(index=['Kennikot','Langenferner']).index),leave_n)
        
        # params.drop(index='Kennikot',inplace=True)
        # params.drop(index='Langenferner',inplace=True)
        names=params.index

        # x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
        # y_data_mean_wind=params['mean_wind_obs']
        # x_data_sensitivity=params[['relief_0.1','Distance from head']]
        # y_data_sensitivity=params['sensitivity']
        # y_data_linsensitivity=params['sensitivity']
        # x_data_delay=params[['rel_distancefromterminus','stddev_1']]

        # x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
        # y_data_mean_wind=params['mean_wind_obs']
        # x_data_sensitivity=params[['relief_0.1','Distance from head']]
        # y_data_sensitivity=params['sensitivity']
        # y_data_linsensitivity=params['sensitivity']
        # x_data_delay=params[['stddev_1', 'rel_distancefromterminus']]
        # y_data_delay=params['delay']

        # After choosing best predictor 
        x_data_mean_wind=params[[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']]
        y_data_mean_wind=params['mean_wind_obs']
        x_data_sensitivity=params[[ 'station_z', 'Z_mean_10km']]
        # x_data_sensitivity=params[[ 'Z_mean_10km']]
        y_data_sensitivity=params['sensitivity']
        y_data_linsensitivity=params[['sensitivity']]
        x_data_delay=params[[ 'Slope_100m']]
        # x_data_delay=params[[ 'glacier_length']]
        y_data_delay=params['delay']


        # x_data_mean_wind=params[['channel_width_at_station', 'stddev_10', 'T_range_continentality']]
        # y_data_mean_wind=params['mean_wind_obs']
        # x_data_sensitivity=params[['Distance from head','glacier_area']]
        # y_data_sensitivity=params['sensitivity']
        # y_data_linsensitivity=params['sensitivity']
        # x_data_delay=params[['stddev_1', 'rel_distancefromhead']]
        # y_data_delay=params['delay']
        names=list(names)
        # for outlier in ['Otemma','Langenferner','Weissee','Corbassiere','Vernatferner','Balcony','Kennikot']:
        #     names.remove(outlier)
        RMSE_ours=[]
        RMSE_era=[]
        params['RGI_region']=[s[0:17] for s in params['Glacier_ID']]
        same_region_stations = params.where(~params['RGI_region'].isin([
            'RGI2000-v7.0-G-15',
            'RGI2000-v7.0-G-14',
            'RGI2000-v7.0-G-13'
            ]))['RGI_region'].dropna().index



        # runfor=same_region_stations
        runfor=names

        for i,name in enumerate(runfor):
            # if i==1:
            #     continue
            # mean wind prediction
            
            same_region_stations = params.where(~params['RGI_region'].isin([
            'RGI2000-v7.0-G-15',
            'RGI2000-v7.0-G-14',
            'RGI2000-v7.0-G-13'
            ]))['RGI_region'].dropna().index
            same_region_stations=[leave_stations]
            
            from sklearn.linear_model import LinearRegression
            Lr=LinearRegression()
            leave=np.append(same_region_stations,['Kennikot'])
            model=Lr.fit(X=x_data_mean_wind.drop(index=leave),y=y_data_mean_wind.drop(index=leave))
 
            beta[leave_n,0,iteration]=model.intercept_
            beta[leave_n,1,iteration]=model.coef_[0]
            beta[leave_n,2,iteration]=model.coef_[1]
            beta[leave_n,3,iteration]=model.coef_[2]
            
            
            x_validate_mean_wind=x_data_mean_wind.loc[[name]]
            if x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]>40:
                x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
                print("Ad-hoc threshold applied")   
            Mean_wind=model.predict(x_validate_mean_wind)


            # sensitivity prediction
            from sklearn.linear_model import LinearRegression
            Lr=LinearRegression()
            leave=np.append(same_region_stations,'Langenferner')
            model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_sensitivity.drop(index=leave))
   
            beta[leave_n,4,iteration]=model.intercept_
            beta[leave_n,5,iteration]=model.coef_[0]
            beta[leave_n,6,iteration]=model.coef_[1]
            
            
            
            
            
            
            
            x_validate_sensitivity=x_data_sensitivity.loc[[name]]
            S=model.predict(x_validate_sensitivity)
            # linear sensitivity prediction
            # from sklearn.linear_model import LinearRegression
            # Lr=LinearRegression()
            # leave=[name]
            # model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_linsensitivity.drop(index=leave))
            # x_validate_sensitivity=x_data_sensitivity.loc[[name]]
            # S_reg=model.predict(x_validate_sensitivity)

            
            
            # delay prediction
            from sklearn.linear_model import LinearRegression
            Lr=LinearRegression()
            leave=np.append(same_region_stations,'Langenferner')
            model=Lr.fit(X=x_data_delay.drop(index=leave),y=y_data_delay.drop(index=leave))
            x_validate_delay=x_data_delay.loc[[name]]
            D=model.predict(x_validate_delay)   

            beta[leave_n,7,iteration]=model.intercept_
            beta[leave_n,8,iteration]=model.coef_[0]

            A=S
            B=-D*S


            D_reg=0
            A_reg=0
            B_reg=0

            # Time series
            name=names[i]
            station_data=pd.read_csv('Universalhourlydatasets//'+name+'.csv')
            station_data['datetime']=pd.to_datetime(station_data['datetime'])
            linear_model=station_data.copy()
            linear_model['temp_model']=linear_model['temp_model']-linear_model['temp_model'].mean()
            linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
            linear_model['ws_model']=linear_model['ws_model']-linear_model['ws_model'].mean()

            lres_wind_speed=np.array(((A*linear_model['temp_model']+B*linear_model['temp_model'].diff())).replace(np.nan,0))+Mean_wind*1
            lreg_wind_speed=station_data['ws'].values*0
            observed_wind_speed=station_data['ws'].values-station_data['ws'].mean()*0
            era5_wind_speed=station_data['ws_model'].values-station_data['ws_model'].mean()*0

            # if not (i==3):
            for l in range(23):
                yodata.append(observed_wind_speed[l])
                ypdata.append(lres_wind_speed[l])
                ypldata.append(lreg_wind_speed[l])
                yedata.append(era5_wind_speed[l])
            RMSE_ours.append(root_mean_squared_error(observed_wind_speed,lres_wind_speed))
            RMSE_era.append(root_mean_squared_error(observed_wind_speed, era5_wind_speed))
            
        beta[leave_n,9,iteration]=np.mean(RMSE_ours)
        print(leave_n,np.mean(RMSE_ours))

# np.savetxt("Applicability.csv",beta)


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: S

Ad-hoc threshold applied
15 0.7680237353508989


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: S

Ad-hoc threshold applied
14 0.7796755562067517


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: S

...

C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: S

Ad-hoc threshold applied
1 0.6900332116165762


C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_14908\276826360.py:110: S

Ad-hoc threshold applied


KeyboardInterrupt: 

In [ ]:
# np.save("beta",beta)

In [21]:
beta=np.load("beta.npy")
fig,ax=fig_row(1,figsize=(5,3))
plt.set_cmap('viridis')
sns.boxenplot(np.transpose(beta[::-1,9,:]),orient='x',outlier_prop=0.05,k_depth='proportion',color='dodgerblue')
plt.scatter(np.arange(16),np.mean(beta[::-1,9,:],axis=1),color='black')
# plt.plot(beta[:,9,:])
plt.xticks(ticks=np.arange(16),labels=28-np.arange(0,16,1)[::-1])
plt.tick_params(axis='y',rotation=90)
plt.xlabel("N")
plt.ylabel(r"RMSE "+r"(ms$^{-1}$)")
plt.legend()
plt.show()
plt.savefig(r"Applicability.pdf", bbox_inches='tight')

C:\Users\krish\AppData\Local\Temp\ipykernel_14908\3978039441.py:11: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [ ]:
# The mean value of the above plot
plt.plot(28-np.arange(16),[x for x in zip((np.mean(beta[:,9,:],axis=1)-0.6863010527536231)/0.6863010527536231*100)])
print([x for x in zip(28-np.arange(16),(np.mean(beta[:,9,:],axis=1)-0.6863010527536231)/0.6863010527536231*100)])
plt.axhline(5,linestyle='--')
plt.axhline(8,linestyle='--')
plt.axhline(10,linestyle='--')
plt.show()

# Table of relative importance of predictors

In [ ]:
betas=pd.read_csv('betas.csv')

In [ ]:
betas['model_parameter']=['mean_wind_obs','mean_wind_obs','mean_wind_obs','mean_wind_obs','sensitivity','sensitivity','sensitivity','delay','delay']

In [ ]:
betas['model_parameter']=['mean_wind_obs','mean_wind_obs','mean_wind_obs','mean_wind_obs','sensitivity','sensitivity','sensitivity','delay','delay']
betas['corr']=betas['SE']
for i in range(len(betas)):
    if betas['predictor'].loc[i]!='const':
        betas['corr'].loc[i]=params[[betas['model_parameter'].loc[i],betas['predictor'].loc[i]]].corr().values[1,0]
betas['t_stat']=abs(betas['Beta']/betas['SE'])
betas.to_csv("TableS3_cryosphere.csv")

# Diurnal prediction of ERA5L good?

In [94]:
with PdfPages("Diurnal_prediction.pdf") as pdf:
    for name in params.index:
        fig, [ax1,] = fig_row(cols=1,figsize=(5,5))
        data = pd.read_csv("Universalhourlydatasets/Predictedhourlydatasets/" + name + ".csv")
        data['datetime']=pd.to_datetime(data['datetime'])
        data.set_index('datetime',inplace=True)
        # # data['temp']=0
        # data=data.groupby(data.index.hour).mean()

        data[['ws_obs','ws_era','ws_pred']]=(data[['ws_obs','ws_era','ws_pred']]-data[['ws_obs','ws_era','ws_pred']].mean())/(data[['ws_obs','ws_era','ws_pred']].max()-data[['ws_obs','ws_era','ws_pred']].min())
        # data[['ws_obs','ws_era','ws_pred']].plot(ax=ax1)
        data.plot.scatter(x='ws_obs',y='ws_pred',color='dodgerblue',ax=ax1)
        data.plot.scatter(x='ws_obs',y='ws_era',color='deeppink',ax=ax1)
        plt.axline((0,0),(1,1),color='black')
        ax1.set_title(name)
        # ax2 = ax1.twinx()
        # data[['wsdir']].plot(ax=ax2, alpha=0.4, zorder=-2, color='gray', legend=False)
        # data[['dgdir']].plot(ax=ax2, alpha=0.4, zorder=-2, color='black',linestyle='--', legend=False)
        # ax2.set_ylim(-180,180)
        plt.show()
        pdf.savefig()
        plt.close(fig)


In [98]:
with PdfPages("Diurnal_prediction_lineplot.pdf") as pdf:
    for name in params.index:
        fig, [ax1,] = fig_row(cols=1,figsize=(5,5))
        data = pd.read_csv("Universalhourlydatasets/Predictedhourlydatasets/" + name + ".csv")
        data['datetime']=pd.to_datetime(data['datetime'])
        data.set_index('datetime',inplace=True)
        # # data['temp']=0
        # data=data.groupby(data.index.hour).mean()

        data[['ws_obs','ws_era','ws_pred']]=(data[['ws_obs','ws_era','ws_pred']]-data[['ws_obs','ws_era','ws_pred']].mean())/(data[['ws_obs','ws_era','ws_pred']].max()-data[['ws_obs','ws_era','ws_pred']].min())
        # data[['ws_obs','ws_era','ws_pred']].plot(ax=ax1)
        data.plot.line(y='ws_obs',color='green',ax=ax1)
        data.plot.line(y='ws_era',color='deeppink',ax=ax1)
        data.plot.line(y='ws_pred',color='dodgerblue',ax=ax1)
        plt.legend()
        ax1.set_title(name)
        # ax2 = ax1.twinx()
        # data[['wsdir']].plot(ax=ax2, alpha=0.4, zorder=-2, color='gray', legend=False)
        # data[['dgdir']].plot(ax=ax2, alpha=0.4, zorder=-2, color='black',linestyle='--', legend=False)
        # ax2.set_ylim(-180,180)
        plt.show()
        pdf.savefig()
        plt.close(fig)

# Figure 1: Relative location of stations

In [22]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Circle
import pandas as pd
import numpy as np

# Load data
params = pd.read_csv("New_bigtable/Universalparams.csv")

# Figure setup
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(9, 3))  # Reduced width
gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1], wspace=0.1)  # Less space between plots

# Axes
ax0 = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())
ax1 = fig.add_subplot(gs[1])

# Panel labels
ax0.text(-0.08, 1.02, 'a.', transform=ax0.transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')
ax1.text(-0.15, 1.02, 'b.', transform=ax1.transAxes,
         fontsize=15, fontweight='bold', va='center', ha='center')

# Map
ax0.coastlines(resolution='110m', linewidth=0.2)
ax0.add_feature(cfeature.LAND, facecolor='0.9')

# Hexbin clustering
hb = ax0.hexbin(params['lon'], params['lat'], gridsize=10, mincnt=1,
                visible=False, transform=ccrs.PlateCarree())
verts = hb.get_offsets()
counts = hb.get_array()

# Circles and counts
for (x, y), c in zip(verts, counts):
    radius = 1.2 * np.log(300 * c / 14)
    dx, dy = (2, 4) if x < 30 else (2, -2)
    circle = Circle((x + dx, y + dy), radius=radius,
                    facecolor='lightcyan', edgecolor='black',
                    linewidth=1, alpha=0.5,
                    transform=ccrs.PlateCarree(), zorder=5)
    ax0.add_patch(circle)
    ax0.text(x + dx, y + dy, str(int(c)),
             ha='center', va='center',
             fontsize=10, color='black', weight='bold',
             zorder=6, transform=ccrs.PlateCarree())

# Repulsion for station markers
lon_orig, lat_orig = params['lon'].values, params['lat'].values
lon_disp, lat_disp = lon_orig.copy(), lat_orig.copy()
np.random.seed(42)
lon_disp += np.random.uniform(-0.01, 0.01, size=len(lon_orig))
lat_disp += np.random.uniform(-0.01, 0.01, size=len(lat_orig))

k_repulsion = 0.01
min_dist = 2
for _ in range(500):
    for i in range(len(lon_disp)):
        for j in range(i + 1, len(lon_disp)):
            dx = lon_disp[i] - lon_disp[j]
            dy = lat_disp[i] - lat_disp[j]
            dist_sq = dx**2 + dy**2
            if dist_sq < min_dist**2:
                dist = np.sqrt(dist_sq) or 1e-6
                force = k_repulsion / dist
                fx = force * dx / dist
                fy = force * dy / dist
                lon_disp[i] += fx
                lon_disp[j] -= fx
                lat_disp[i] += fy
                lat_disp[j] -= fy

# Arrows from original to displaced
for lon0, lat0, lon1, lat1 in zip(lon_orig, lat_orig, lon_disp, lat_disp):
    ax0.annotate('', xy=(lon1, lat1), xytext=(lon0, lat0),
                 xycoords=ccrs.PlateCarree()._as_mpl_transform(ax0),
                 textcoords=ccrs.PlateCarree()._as_mpl_transform(ax0),
                 zorder=0)

# Station markers
ax0.scatter(lon_disp, lat_disp,
            s=20, color='deeppink', edgecolors='black', linewidth=0.5,
            transform=ccrs.PlateCarree(), zorder=2)

# Plotting ax1
params = params.sample(frac=1).reset_index(drop=True)
x = params['rel_distancefromcenterline'] * params['left_right']

ax1.fill_between(np.linspace(-0.5, 0.5, 100), 0, 1, color='lightcyan', zorder=-100)
ax1.fill_between(np.linspace(-2.5, 2.5, 100), -2, 0, color='lightcyan', zorder=-100)

ax1.scatter(x, params['rel_distancefromhead'],
            edgecolor='black', s=60, linewidth=0.5, color="deeppink")

ax1.set_xlim(-2.5, 2.5)
ax1.set_ylim(bottom=0)
ax1.set_facecolor("#EFDECD")
ax1.set_xlabel("Normalised distance from centerline")
ax1.set_ylabel("Normalised distance from head")
ax1.yaxis.tick_right()
ax1.yaxis.set_label_position('right')
ax1.tick_params(axis='y', labelrotation=90)

# Save and show
plt.savefig("Station_global_distribution.pdf", bbox_inches='tight')
plt.show()


In [23]:
params['Down_glacier_direction'].loc['Drangdrung']

KeyError: 'Drangdrung'

In [ ]:
import matplotlib.pyplot as plt

# Count occurrences of each value
counts = params['left_right'].value_counts().sort_index()

# Define labels (optional: replace -1 and 1 with readable text)
labels = ['Left (-1)', 'Right (1)']

# Plot pie chart
plt.pie(counts, labels=labels, autopct='%1.1f%%', startangle=90)
plt.axis('equal')  # Equal aspect ratio ensures it's a circle
plt.title('Left vs Right Distribution')
plt.show()


In [ ]:
set_plot_style()
glacier_bed=np.linspace(-0.01,1.,1000)
glacier_surface=np.exp(-glacier_bed**0.25)*(1-glacier_bed)**0.5
def elevation(c):
    return [0.5*np.exp(-(c+0.05)**0.25)*(1-(c+0.05))**0.5 if c<1.05 else 0.01 for c in c]
def bed_elevation(c):
    return (1-c-0.05)*0.15
plt.figure(figsize=(10,5))

plt.fill_between(glacier_bed,bed_elevation(glacier_bed),elevation(glacier_bed),color="#A0FEFE")
plt.fill_between(glacier_bed,bed_elevation(glacier_bed),0,color="#C9B185C6")
plt.grid(False)
plt.plot(glacier_bed,bed_elevation(glacier_bed),color="#000000")
plt.plot(glacier_bed,elevation(glacier_bed),color="#000000")
params.sort_values(by='rel_distancefromhead',inplace=True)
prev=0
count=1
switch=1
plotted_name=""
# params.set_index('glacier_name',inplace=True)
cmap=plt.colormaps['coolwarm']
for name in params.index:
    
    if np.abs(prev-params['rel_distancefromhead'].loc[name])>0.05:
        
        plt.scatter(params['rel_distancefromhead'].loc[name],elevation([params['rel_distancefromhead'].loc[name]]),s=25,marker='D',edgecolors='black',color=cmap(params['rel_distancefromcenterline'].loc[name]))
        plotted_name=name
        prev=params['rel_distancefromhead'].loc[name]
        switch=1
        count=1
        print("first point",name)
    else:
        
        plt.scatter(prev,elevation([prev])[0]+0.015*count,s=25,marker='D',edgecolors='black',color=cmap(params['rel_distancefromcenterline'].loc[name]))    
        plotted_name=name
        print("addontop",name)
        count+=1    
        switch=0
    if name!=plotted_name:
        print(name)

# plt.text()

plt.xlim(-0.01,1.4)
plt.ylim(0,0.5)

In [ ]:
params.plot.scatter(x='rel_distancefromhead',y=params['rel_distancefromcenterline']*params['leftright'])

In [ ]:
import numpy as np
from scipy.stats import pearsonr

x = params['sensitivity']
y = params['delay']

# covariance:
cov_xy = np.cov(x, y, bias=True)[0,1]
print("Covariance:", cov_xy)

# Pearson r and p-value:
r, p = pearsonr(x, y)
print("r:", r, "p-value:", p)


In [ ]:
params.plot.scatter(x='sensitivity',y='rel_distancefromhead',c='continentality')
plt.show()

In [ ]:
params['data_length_days'].median()

In [ ]:
import geopandas as gpd 
import cartopy.crs as ccrs
import cartopy.feature as cfeature

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='110m', linewidth=0.7)
# ax.add_feature(cfeature.BORDERS, linewidth=0.5)

# Optionally add land for better contrast
ax.add_feature(cfeature.LAND, facecolor='lightgray')
params.plot.scatter(x='lon',y='lat',ax=ax,s=10,zorder=10,edgecolor='black',linewidth=0.5,color='deeppink')
plt.show()

In [ ]:
import geopandas as gpd 
import cartopy.crs as ccrs
import cartopy.feature as cfeature

ax = plt.axes(projection=ccrs.PlateCarree())

ax.coastlines(resolution='110m', linewidth=0.7)
# ax.add_feature(cfeature.BORDERS, linewidth=0.5)

# Optionally add land for better contrast
ax.add_feature(cfeature.LAND, facecolor='lightgray')
params.plot.scatter(x='lon',y='lat',ax=ax,s=30,zorder=10,edgecolor='black',linewidth=0.5,color='deeppink')
ax.set_extent([75,90,25,35], crs=ccrs.PlateCarree())

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Circle

# Create Cartopy map
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
# ESRI shaded relief: detailed, fast
# Optional extent (zoom)
extent = [params['lon'].min(), params['lon'].max(), params['lat'].min(), params['lat'].max()]
# ax.set_extent(extent, crs=ccrs.PlateCarree())

# Add map features
ax.coastlines(resolution='110m',linewidth=0.2)
ax.add_feature(cfeature.LAND, facecolor='0.9')
# ax.add_feature(cfeature.BORDERS, linewidth=0.5)
import cartopy.io.img_tiles as cimgt




# Step 1: Use hexbin (but don’t show it) just to get bin centers + counts

hb = ax.hexbin(
    params['lon'], params['lat'],
    gridsize=10,
    mincnt=1,
    visible=False,  # Don’t draw
    transform=ccrs.PlateCarree()
)

# Step 2: Get bin centers and counts
verts = hb.get_offsets()
counts = hb.get_array()
max_count = max(counts)

# Step 3: Draw circles and labels
for (x, y), c in zip(verts, counts):
    radius = np.log(200*c/14)  # scale radius to count
    if x<30:
        circle = Circle(
            (x+2, y+4),
            radius=radius,
            facecolor='lightgray',
            edgecolor='black',
            linewidth=1,
            alpha=0.9,
            transform=ccrs.PlateCarree(),
            zorder=5
        )
        ax.add_patch(circle)
        
        ax.text(
        x+2, y+4, str(int(c)),
        ha='center', va='center',
        fontsize=10,
        color='black',
        weight='bold',
        zorder=6,
        transform=ccrs.PlateCarree()
        )
    else:
        circle = Circle(
            (x+2, y-2),
            radius=radius,
            facecolor='lightgray',
            edgecolor='black',
            linewidth=1,
            alpha=0.9,
            transform=ccrs.PlateCarree(),
            zorder=5
        )
        ax.add_patch(circle)
        ax.text(
        x+2, y-2, str(int(c)),
        ha='center', va='center',
        fontsize=10,
        color='black',
        weight='bold',
        zorder=6,
        transform=ccrs.PlateCarree()
        )
    
    # Add count label inside circle
    
    
import numpy as np

# Original locations
lon_orig = params['lon'].values
lat_orig = params['lat'].values

# Start with slight random displacement
np.random.seed(42)
lon_disp = lon_orig + np.random.uniform(-0.01, 0.01, size=len(lon_orig))
lat_disp = lat_orig + np.random.uniform(-0.01, 0.01, size=len(lat_orig))

# Parameters
k_repulsion = 0.01     # strength of repulsive force
min_dist = 2       # minimum allowed distance between points
n_iter = 500            # number of iterations

# Force-directed adjustment loop
for _ in range(n_iter):
    for i in range(len(lon_disp)):
        for j in range(i + 1, len(lon_disp)):
            dx = lon_disp[i] - lon_disp[j]
            dy = lat_disp[i] - lat_disp[j]
            dist_sq = dx**2 + dy**2

            if dist_sq < min_dist**2:
                dist = np.sqrt(dist_sq) or 1e-6
                force = k_repulsion / dist
                fx = force * dx / dist
                fy = force * dy / dist

                lon_disp[i] += fx
                lon_disp[j] -= fx
                lat_disp[i] += fy
                lat_disp[j] -= fy

# Pin lines from true location to displaced
for lon0, lat0, lon1, lat1 in zip(lon_orig, lat_orig, lon_disp, lat_disp):
    ax.annotate(
        '', xy=(lon1, lat1), xytext=(lon0, lat0),
        xycoords=ccrs.PlateCarree()._as_mpl_transform(ax),
        textcoords=ccrs.PlateCarree()._as_mpl_transform(ax),
        zorder=0
    )


# Jittered markers after repulsion
ax.scatter(
    lon_disp, lat_disp,
    s=20, color='deeppink', edgecolors='black', linewidth=0.5,
    transform=ccrs.PlateCarree(),
    zorder=2
)
# ax.scatter(
#     lon_orig, lat_orig,
#     s=2, color='black', edgecolors='black', linewidth=0.5,
#     transform=ccrs.PlateCarree(),
#     zorder=2
# )


plt.savefig("Station_global_distribution.pdf")
plt.show()


# Figure 1: locations of 28 stations around the world as well as within a normalised glacier

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.patches import Circle
import pandas as pd
import numpy as np

# Load data
params = pd.read_csv("New_bigtable/Universalparams.csv")

# Figure setup
import matplotlib.gridspec as gridspec
fig = plt.figure(figsize=(8, 4))  # Reduced width
gs = gridspec.GridSpec(1, 2, width_ratios=[2.5, 1], wspace=0.1)  # Less space between plots

# Axes

fig1=plt.figure(figsize=(6, 4))
fig2=plt.figure(figsize=(2, 4))
ax0 = fig1.add_subplot( projection=ccrs.PlateCarree())
ax1 = fig2.add_subplot()
# for spine in ax1.spines.values():
#     spine.set_edgecolor('red')
#     spine.set_linewidth(2)
# for spine in ax0.spines.values():
#     spine.set_edgecolor('red')
#     spine.set_linewidth(2)
# for ax in [ax0, ax1]:
#     bb = ax.get_position()
#     fig.patches.append(plt.Rectangle(
#         (bb.x0, bb.y0), bb.width, bb.height,
#         transform=fig.transFigure, fill=False, edgecolor='red', linewidth=2
#     ))

# Panel labels
ax0.text(0.03, 0.95, 'a.', transform=ax0.transAxes,
         fontsize=10, fontweight='bold', va='center', ha='center')
ax1.text(0.08, 0.95, 'b.', transform=ax1.transAxes,
         fontsize=10, fontweight='bold', va='center', ha='center')
ax1.set_aspect(4)

# Map
ax0.coastlines(resolution='110m', linewidth=0.2)
ax0.add_feature(cfeature.LAND, facecolor='0.9')
ax0.set_ylim(-60,85)
# Hexbin clustering
hb = ax0.hexbin(params['lon'], params['lat'], gridsize=10, mincnt=1,
                visible=False, transform=ccrs.PlateCarree())
verts = hb.get_offsets()
counts = hb.get_array()

# Circles and counts
for (x, y), c in zip(verts, counts):
    radius = 1.2 * np.log(300 * c / 14)
    dx, dy = (2, 4) if x < 30 else (2, -2)
    circle = Circle((x + dx, y + dy), radius=radius,
                    facecolor='lightcyan', edgecolor='black',
                    linewidth=0.5, alpha=0.5,
                    transform=ccrs.PlateCarree(), zorder=5)
    ax0.add_patch(circle)
    ax0.text(x + dx, y + dy, str(int(c)),
             ha='center', va='center',
             fontsize=10, color='black', weight='regular',
             zorder=6, transform=ccrs.PlateCarree())

# Repulsion for station markers
lon_orig, lat_orig = params['lon'].values, params['lat'].values
lon_disp, lat_disp = lon_orig.copy(), lat_orig.copy()
np.random.seed(42)
lon_disp += np.random.uniform(-0.01, 0.01, size=len(lon_orig))
lat_disp += np.random.uniform(-0.01, 0.01, size=len(lat_orig))

k_repulsion = 0.01
min_dist = 2
for _ in range(500):
    for i in range(len(lon_disp)):
        for j in range(i + 1, len(lon_disp)):
            dx = lon_disp[i] - lon_disp[j]
            dy = lat_disp[i] - lat_disp[j]
            dist_sq = dx**2 + dy**2
            if dist_sq < min_dist**2:
                dist = np.sqrt(dist_sq) or 1e-6
                force = k_repulsion / dist
                fx = force * dx / dist
                fy = force * dy / dist
                lon_disp[i] += fx
                lon_disp[j] -= fx
                lat_disp[i] += fy
                lat_disp[j] -= fy

# Arrows from original to displaced
for lon0, lat0, lon1, lat1 in zip(lon_orig, lat_orig, lon_disp, lat_disp):
    ax0.annotate('', xy=(lon1, lat1), xytext=(lon0, lat0),
                 xycoords=ccrs.PlateCarree()._as_mpl_transform(ax0),
                 textcoords=ccrs.PlateCarree()._as_mpl_transform(ax0),
                 zorder=0)

# Station markers
ax0.scatter(lon_disp, lat_disp,
            s=20, color='deeppink', edgecolors='black', linewidth=0.5,
            transform=ccrs.PlateCarree(), zorder=2)

# Plotting ax1
params = params.sample(frac=1).reset_index(drop=True)
x = params['rel_distancefromcenterline'] * params['left_right']

ax1.fill_between(np.linspace(-0.5, 0.5, 100), 0, 1, color='lightcyan', zorder=-100)
ax1.fill_between(np.linspace(-2.5, 2.5, 100), -2, 0, color='lightcyan', zorder=-100)

ax1.scatter(x, params['rel_distancefromhead'],
            edgecolor='black', s=60, linewidth=0.5, color="deeppink")

ax1.set_xlim(-2.1, 2.1)
ax1.set_yticks(np.arange(0.2,1.5,0.4))
ax1.set_yticks(np.arange(0,1.5,0.4),minor=True)
ax1.set_ylim(bottom=0)
ax1.set_facecolor("#EFDECD")
ax1.set_xlabel("Normalised Y")
ax1.set_ylabel("Normalised X")
ax1.yaxis.tick_right()
ax1.yaxis.set_label_position('right')
ax1.tick_params(axis='y', labelrotation=90)

# Save and show
# plt.savefig("relative_station_locations_1.pdf", bbox_inches='tight',fig=fig1)
# plt.savefig("relative_station_locations_2.pdf", bbox_inches='tight',fig=fig2,)
fig1.savefig("Station_global_distribution1.pdf", bbox_inches='tight')
fig2.savefig("Station_global_distribution2.pdf", bbox_inches='tight')

plt.show()



# Checking variability of model parameters over different years

In [ ]:
from sklearn.metrics import root_mean_squared_error
from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load parameters and preprocess
params = pd.read_csv(r'New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)
# Station names
names=params.index

yearly_variability_df=pd.DataFrame(columns=['Station','u','u_dash','days_of_data','t_dash'])

for name in names:
    station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
    station_data['datetime'] = pd.to_datetime(station_data['datetime'])
    station_data[station_data['ws']<-0.1]=np.nan
    station_data.dropna(inplace=True)
    station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
    station_data['year'] = station_data['datetime'].dt.year
    station_data.set_index('datetime', inplace=True)
    station_data_hourly = station_data.copy()
    station_data_hourly.dropna(inplace=True)
    station_data_hourly = station_data_hourly.resample('1h').mean()
    # station_data_hourly=station_data_hourly.interpolate(method='time')
    station_data = station_data_hourly.reset_index()
    # print(len(station_data_hourly.index.year.unique()))
    no_of_years_of_data=len(station_data_hourly.index.year.unique())
    years_of_data=station_data_hourly.index.year.unique()
    if(no_of_years_of_data>1):
        # print(name,years_of_data)
        for year in years_of_data:
            station_data_year=station_data[(station_data['datetime'].dt.year == year)]
            yearly_variability_df.loc[len(yearly_variability_df)]={'Station':name,'u':params['mean_wind_obs'].loc[name],'u_dash':np.abs(station_data_year['ws'].mean()-params['mean_wind_obs'].loc[name])/params['mean_wind_obs'].loc[name],'days_of_data':len(station_data_year['datetime'].dt.date.unique()),'t_dash':station_data_year['temp'].mean()}
            print(len(station_data_year['datetime'].dt.date.unique()))



214
366
365
365
182


C:\Users\krish\AppData\Local\Temp\ipykernel_8924\1156880633.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


198
366
269
214
365
365
274
214
365
239
214
222
214
365
273
150
365
225
150
365
272
150
365
244
150
365
238
214
365
365
273
214
365
365
365
253
214
366
365
365
185
214
365
365
272
214
273
214
365
365
273


In [ ]:
import seaborn as sns
set_plot_style()
fig=fig_row(1,figsize=(8,4))
sns.boxplot(data=yearly_variability_df,x='Station',y='u_dash',color='lightpink',linecolor='black')
sns.scatterplot(x=plt.gca().get_xticks(),y=yearly_variability_df['u_dash'].groupby(yearly_variability_df['Station']).mean(),s=20,color='black',zorder=10)
plt.xticks(rotation=90)
plt.ylabel(r"$\dfrac{d\bar{u}^{obs}}{\bar{u}^{obs}}$")
plt.xticks(rotation=45)
# plt.yscale('log')
plt.show()
plt.savefig("Interannual.pdf",bbox_inches='tight')

In [39]:
yearly_variability_df['t_dash'].groupby(yearly_variability_df['Station']).var().sort_values()

Station
Morimoto        0.002937
Trakarding      0.005199
Yala            0.044700
Lirung          0.087952
Schiaparelli    0.105066
Weissee         0.144854
Camp2           0.251601
Place4          0.293271
Kalapathar      0.405968
Place2          0.514978
Place1          0.656940
Southcol        1.172563
Langenferner    1.989181
Place3          2.233855
Pyramid         2.588290
Djankuat        5.161322
Name: t_dash, dtype: float64

In [69]:
plt.set_cmap('coolwarm_r')
plt.scatter(yearly_variability_df['u'],yearly_variability_df['u_dash'],c=yearly_variability_df['days_of_data'])
# plt.scatter(yearly_variability_df['u'],yearly_variability_df['u'],marker='x',color='k',zorder=9)
plt.colorbar(label='mean_temperature_year')
plt.axline((0,0),(7,7))
plt.xlabel("Overall mean wind parameter")
plt.ylabel("yearly mean wind parameter")
plt.show()

In [ ]:
yearly_variability_df1=yearly_variability_df[(yearly_variability_df['Station']!='Djankuat')*(yearly_variability_df['Station']!='Southcol')]
for station in yearly_variability_df1['Station'].unique():
    yearly_variability_df1_=yearly_variability_df1[yearly_variability_df1['Station']==station]
    print(station,root_mean_squared_error(yearly_variability_df1_['u'],yearly_variability_df1_['u_dash']))

In [ ]:
%matplotlib qt
sns.scatterplot(x=yearly_variability_df['t_dash'],y=yearly_variability_df['u_dash'],hue=yearly_variability_df['Station'])
plt.xlabel("temperature anomaly")
plt.ylabel("mean wind anomaly")
plt.show()

# Whats going on in djankuat and southcol?

In [60]:
df=pd.read_csv(r"Universaldatasets\Southcol.csv")
df['datetime']=pd.to_datetime(df['datetime'])
df.set_index('datetime',inplace=True)

In [62]:
df['ws'].plot()
plt.show()

In [59]:
import seaborn as sns
sns.jointplot(data=df,x='ws',y='wdir',hue='temp',s=1)
plt.show()

In [364]:
for name in params.index:
    df = pd.read_csv("Universaldatasets_smallerdomain/" + name + ".csv")
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Get unique dates
    unique_dates = df['datetime'].dt.date.unique()
    unique_dates.sort()

    start_date = unique_dates[0]
    end_date = unique_dates[-1]
    num_days = params['data_length_days'].loc[name]

    print(f"{start_date} to {end_date} ({num_days} days)")


2022-07-06 to 2022-09-18 (74 days)
2019-06-01 to 2019-09-30 (122 days)
2015-07-27 to 2015-09-28 (64 days)
2022-06-01 to 2022-07-29 (59 days)
2019-06-01 to 2023-07-01 (518 days)
2022-08-16 to 2022-09-29 (45 days)


C:\Users\krish\AppData\Local\Temp\ipykernel_14452\2367327989.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime'])


2022-06-07 to 2022-09-29 (86 days)
2007-06-17 to 2009-09-30 (231 days)
2023-06-01 to 2023-09-23 (98 days)
2012-06-01 to 2012-09-30 (121 days)
2009-06-01 to 2012-09-30 (457 days)
2019-05-31 to 2019-08-21 (82 days)
2013-06-01 to 2015-08-27 (332 days)
2018-06-01 to 2019-08-10 (186 days)
2017-06-01 to 2019-09-30 (365 days)
2006-08-04 to 2008-08-12 (206 days)
2006-08-04 to 2008-09-28 (266 days)
2006-08-04 to 2008-08-31 (241 days)
2006-08-04 to 2008-08-25 (189 days)
2009-06-01 to 2012-09-29 (473 days)
2017-06-29 to 2017-09-29 (92 days)
2016-05-31 to 2020-09-09 (586 days)
2019-06-01 to 2023-07-04 (419 days)
2016-06-01 to 2019-09-29 (487 days)
2007-07-10 to 2007-09-06 (58 days)
2007-07-10 to 2007-09-06 (58 days)
2017-06-01 to 2018-09-30 (243 days)
2016-06-01 to 2019-09-30 (464 days)


# Model with betas and ses

In [44]:
mean_wind_predictors=[ 'relief_1km', 'relief_5km', 'aspect_ratio_stddev_1']
sensitivity_predictors=[ 'station_z', 'Z_mean_10km']
delay_predictors=[ 'glacier_length']

x1_data_mean_wind=params[mean_wind_predictors]
y1_data_mean_wind=params['mean_wind_obs']
x1_data_sensitivity=params[sensitivity_predictors]
y1_data_sensitivity=params['sensitivity']
y1_data_linsensitivity=params[['sensitivity']]
x1_data_delay=params[delay_predictors]
y1_data_delay=params['delay']

In [45]:
import statsmodels.api as sm
import pandas as pd
params=pd.read_csv('New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)


x_data_mean_wind=x1_data_mean_wind.copy()
y_data_mean_wind=y1_data_mean_wind.copy()
x_data_sensitivity=x1_data_sensitivity.copy()
y_data_sensitivity=y1_data_sensitivity.copy()
y_data_linsensitivity=y1_data_linsensitivity.copy()
x_data_delay=x1_data_delay.copy()
y_data_delay=y1_data_delay.copy()

leave_u=['Kennikot']
leave_S=['Langenferner']
leave_D=['Langenferner']

x_data_mean_wind.drop(leave_u,inplace=True)
y_data_mean_wind.drop(leave_u,inplace=True)
x_data_sensitivity.drop(leave_S,inplace=True)
y_data_sensitivity.drop(leave_S,inplace=True)
x_data_delay.drop(leave_D,inplace=True)
y_data_delay.drop(leave_D,inplace=True)


# beta 1 and beta 2
x_data_mean_wind=sm.add_constant(x_data_mean_wind)
x_data_sensitivity=sm.add_constant(x_data_sensitivity)
x_data_delay=sm.add_constant(x_data_delay)

model_u = sm.OLS(y_data_mean_wind, x_data_mean_wind).fit()
model_S = sm.OLS(y_data_sensitivity, x_data_sensitivity).fit()
model_D = sm.OLS(y_data_delay, x_data_delay).fit()

beta1=model_u.params['const']
d_beta1=model_u.bse['const']

beta2=model_u.params[mean_wind_predictors[0]]
d_beta2=model_u.bse[mean_wind_predictors[0]]

beta3=model_u.params[mean_wind_predictors[1]]
d_beta3=model_u.bse[mean_wind_predictors[1]]

beta4=model_u.params[mean_wind_predictors[2]]
d_beta4=model_u.bse[mean_wind_predictors[2]]

beta5=model_S.params['const']
d_beta5=model_S.bse['const']

beta6=model_S.params[sensitivity_predictors[0]]
d_beta6=model_S.bse[sensitivity_predictors[0]]

beta7=model_S.params[sensitivity_predictors[1]]
d_beta7=model_S.bse[sensitivity_predictors[1]]

beta8=model_D.params['const']
d_beta8=model_D.bse['const']

beta9=model_D.params[delay_predictors[0]]
d_beta9=model_D.bse[delay_predictors[0]]

# beta9=model_D.params['stddev_1']
# d_beta9=model_D.bse['stddev_1']

betas=[beta1,beta2,beta3,beta4,beta5,beta6,beta7,beta8,beta9]
d_betas=[d_beta1,d_beta2,d_beta3,d_beta4,d_beta5,d_beta6,d_beta7,d_beta8,d_beta9]

betas=pd.DataFrame(data={'predictor':list(x_data_mean_wind.columns)+list(x_data_sensitivity.columns)+list(x_data_delay.columns),'Beta':betas,'SE':d_betas})

betas['model_parameter']=['mean_wind_obs','mean_wind_obs','mean_wind_obs','mean_wind_obs','sensitivity','sensitivity','sensitivity','delay','delay']
betas['corr']=betas['SE']
for i in range(len(betas)):
    if betas.loc[i,'predictor']!='const':
        betas.loc[i,'corr']=params[[betas.loc[i,'model_parameter'],betas.loc[i,'predictor']]].corr().values[1,0]
betas['t_stat']=abs(betas['Beta']/betas['SE'])


In [23]:
betas

,predictor,Beta,SE,model_parameter,corr,t_stat
0,const,2.503347,0.510583,mean_wind_obs,0.510583,4.902917
1,relief_1km,0.004518,0.000802,mean_wind_obs,-0.077739,5.633185
2,relief_5km,-0.001567,0.000253,mean_wind_obs,-0.421250,6.201579
3,aspect_ratio_stddev_1,0.112337,0.019797,mean_wind_obs,-0.048599,5.674334
4,const,0.133004,0.094004,sensitivity,0.094004,1.414871
5,station_z,-0.000169,0.000054,sensitivity,0.110555,3.127260
6,Z_mean_10km,0.000217,0.000060,sensitivity,0.322800,3.616306
7,const,0.734495,0.493925,delay,0.493925,1.487058
8,Slope_100m,1.928519,1.453179,delay,0.235539,1.327104


In [ ]:

from matplotlib.backends.backend_pdf import PdfPages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error,r2_score
import seaborn as sns

 
set_plot_style()

# names=[
# 'arolla',
# 'langenferner',
# 'schiaparelli',
# 'Trakarding',
# 'yala',
# 'lirung',
# 'satopanth',
# 'wm2',
# 'pm3',
# 'pm2',
# 'pm1',
# 'djankuat',
# 'chotashigri',
# 'wm1',
# 'drangdrung',
# 'pyramid',
# 'bishop',
# # 'weisssee',
# 'hef',
# # 'namche',
# # 'balcony',
# 'camp2',
# # 'pm4',
# # 'southcol',
# # 'Denalipass',
# 'kalapathar',
# 'bellavista',
# 'millwitzkeez',
# 'venedegerkees',
# 'vernatferner',
# 'otemma',
# 'corbassiere',
# 'kennicot'
# ]
# Leave one out cross validation


params_stats=pd.read_csv('params_stats.csv')
params_stats.set_index('glacier_name',inplace=True)

rmse_lres=[]
rmse_lreg=[]
rmse_era5=[]

yodata=[]
ypdata=[]
ypldata=[]
yedata=[]
yperror=[]
yoerror=[]

params=pd.read_csv(r'New_bigtable/Universalparams.csv')
params.set_index('glacier_name',inplace=True)

# params.drop(index='Kennikot',inplace=True)
# params.drop(index='Langenferner',inplace=True)
names=params.index

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['rel_distancefromterminus','stddev_1']]

# x_data_mean_wind=params[[ 'aspect_ratio_21','relief_20']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['relief_0.1','Distance from head']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromterminus']]
# y_data_delay=params['delay']

x_data_mean_wind=x1_data_mean_wind.copy()
y_data_mean_wind=y1_data_mean_wind.copy()
x_data_sensitivity=x1_data_sensitivity.copy()
y_data_sensitivity=y1_data_sensitivity.copy()
y_data_linsensitivity=y1_data_linsensitivity.copy()
x_data_delay=x1_data_delay.copy()
y_data_delay=y1_data_delay.copy()


# x_data_mean_wind=params[['channel_width_at_station', 'stddev_10', 'T_range_continentality']]
# y_data_mean_wind=params['mean_wind_obs']
# x_data_sensitivity=params[['Distance from head','glacier_area']]
# y_data_sensitivity=params['sensitivity']
# y_data_linsensitivity=params['sensitivity']
# x_data_delay=params[['stddev_1', 'rel_distancefromhead']]
# y_data_delay=params['delay']
names=list(names)
# for outlier in ['Otemma','Langenferner','Weissee','Corbassiere','Vernatferner','Balcony','Kennikot']:
#     names.remove(outlier)
RMSE_ours=[]
RMSE_era=[]
params['RGI_region']=[s[0:17] for s in params['Glacier_ID']]
same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index


# runfor=same_region_stations
runfor=names
for i,name in enumerate(runfor):
    # if i==1:
    #     continue
    # mean wind prediction
    
    same_region_stations = params.where(~params['RGI_region'].isin([
    'RGI2000-v7.0-G-15',
    'RGI2000-v7.0-G-14',
    'RGI2000-v7.0-G-13'
    ]))['RGI_region'].dropna().index
    same_region_stations=[]
    
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,['Kennikot'])
    model=Lr.fit(X=x_data_mean_wind.drop(index=leave),y=y_data_mean_wind.drop(index=leave))
    x_validate_mean_wind=x_data_mean_wind.loc[[name]]
    if name=='Kennikot':
        x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
        # x_validate_mean_wind['aspect_ratio_relief_1'].loc[name]=0
        # x_validate_mean_wind['CW'].loc[name]=0
        
    print("Ad-hoc threshold applied")   
    Mean_wind=model.predict(x_validate_mean_wind)


    # sensitivity prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_sensitivity.drop(index=leave))
    x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    S=model.predict(x_validate_sensitivity)

    # linear sensitivity prediction
    # from sklearn.linear_model import LinearRegression
    # Lr=LinearRegression()
    # leave=[name]
    # model=Lr.fit(X=x_data_sensitivity.drop(index=leave),y=y_data_linsensitivity.drop(index=leave))
    # x_validate_sensitivity=x_data_sensitivity.loc[[name]]
    # S_reg=model.predict(x_validate_sensitivity)

    # delay prediction
    from sklearn.linear_model import LinearRegression
    Lr=LinearRegression()
    leave=np.append(same_region_stations,'Langenferner')
    model=Lr.fit(X=x_data_delay.drop(index=leave),y=y_data_delay.drop(index=leave))
    x_validate_delay=x_data_delay.loc[[name]]
    D=model.predict(x_validate_delay)   


    A=S
    B=-D*S


    D_reg=0
    A_reg=0
    B_reg=0

    # Time series
    name=names[i]
    station_data=pd.read_csv('Universalhourlydatasets//'+name+'.csv')
    station_data['datetime']=pd.to_datetime(station_data['datetime'])
    linear_model=station_data.copy()
    linear_model['temp_model']=linear_model['temp_model']-linear_model['temp_model'].mean()
    linear_model['ws']=linear_model['ws']-linear_model['ws'].mean()
    linear_model['ws_model']=linear_model['ws_model']-linear_model['ws_model'].mean()


    lres_wind_speed=np.array(((A*linear_model['temp_model']+B*linear_model['temp_model'].diff())).replace(np.nan,0))+Mean_wind*1
    lreg_wind_speed=station_data['ws'].values*0
    observed_wind_speed=station_data['ws'].values-station_data['ws'].mean()*0
    era5_wind_speed=station_data['ws_model'].values-station_data['ws_model'].mean()*0
    linear_model['ws_pred']=lres_wind_speed
    S_pred=params_stats['S_pred'].loc[name]
    D_pred=params_stats['D_pred'].loc[name]
    U_pred=params_stats['U_pred'].loc[name]
    
    se_S_pred=params_stats['se_S_pred'].loc[name]
    se_D_pred=params_stats['se_D_pred'].loc[name]
    se_U_pred=params_stats['se_U_pred'].loc[name]
    station_data = pd.read_csv(f'Universaldatasets_smallerdomain/{name}.csv')
    station_data['datetime'] = pd.to_datetime(station_data['datetime'])
    station_data[station_data['ws']<-0.1]=np.nan
    station_data.dropna(inplace=True)
    station_data=station_data[(station_data['datetime'].dt.month > 5) & (station_data['datetime'].dt.month < 10)]
    station_data['year'] = station_data['datetime'].dt.year
    year_with_max_data = station_data['year'].value_counts().idxmax()
    station_data.set_index('datetime', inplace=True)
    station_data_hourly = station_data
    station_data_hourly.dropna(inplace=True)
    station_data_hourly = station_data_hourly.resample('1h').mean()
    # station_data_hourly=station_data_hourly.interpolate(method='time')
    station_data = station_data_hourly.reset_index()
    error_linear_model=station_data_hourly.groupby(station_data_hourly.index.hour).std()/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count())
    error_linear_model=error_linear_model[1:]
    linear_model['dt']=linear_model['temp'].diff()
    linear_model=linear_model.dropna()
    linear_model['dtemp']=station_data_hourly.groupby(station_data_hourly.index.hour).std()['temp']/np.sqrt(station_data_hourly.groupby(station_data_hourly.index.hour).count()['temp'])
    linear_model['ddt']=2*linear_model['dtemp']
    linear_model['d_ws_pred']=np.sqrt((se_U_pred**2)+(S_pred)**2*linear_model['dtemp']+(linear_model['temp']-D_pred*linear_model['dt'])**2*se_S_pred**2+(S_pred**2*D_pred**2*linear_model['dtemp']**2)+(S_pred*linear_model['dt'])**2*se_D_pred**2)
    

    # if not (i==3):
    for l in range(23):
        yodata.append(observed_wind_speed[l])
        ypdata.append(lres_wind_speed[l])
        ypldata.append(lreg_wind_speed[l])
        yedata.append(era5_wind_speed[l])
        yperror.append(linear_model['d_ws_pred'].iloc[l])
        yoerror.append(error_linear_model['ws'].iloc[l])
    RMSE_ours.append(root_mean_squared_error(observed_wind_speed,lres_wind_speed))
    RMSE_era.append(root_mean_squared_error(observed_wind_speed, era5_wind_speed))
    # linear_model['ws']+=
    save_df=pd.DataFrame(columns=['ws_obs','ws_pred','ws_era','temp'])
    save_df['datetime']=np.arange(24)
    save_df['ws_obs']=observed_wind_speed
    save_df['ws_pred']=lres_wind_speed
    save_df['ws_era']=era5_wind_speed
    save_df['temp_era']=station_data['temp'].groupby(station_data.datetime.dt.hour).mean()
    # save_df['']
    
    
    save_df.to_csv('Universalhourlydatasets/Predictedhourlydatasets/'+name+'.csv')
jaja=pd.DataFrame()

jaja['u_era5']=yedata#-jaja['u_obs']
jaja['u_S']=ypldata#-jaja['u_obs']
jaja['u_SD']=ypdata
jaja['u_obs']=yodata
jaja['u_error']=yperror
jaja['u_oerror']=yoerror
jaja['glacier_number']=np.arange(23*len(runfor))//23

pd.DataFrame(RMSE_ours).to_csv("RMSE_ours.csv")
pd.DataFrame(RMSE_era).to_csv("RMSE_era5.csv")


# import scienceplots

# plt.style.use(['science','scatter'])

# plt.rcParams["font.sans-serif"] = ["Nimbus Sans L"]
# plt.rcParams["font.weight"] = "bold"
# plt.rcParams["axes.labelweight"] = "bold"
# plt.rcParams["axes.titleweight"] = "bold"

# fig,ax=fig_row(1)
# plt.axes(ax[0])
# plt.axline((0,0),(1,1),color='black',linewidth=0.5,linestyle='-')
# ax1 = fig.add_axes([0.18,0.55,0.2,0.2])
# plt.axhline(0,color='black',linewidth=1,linestyle='--')
# plt.vlines(jaja['u_obs'],jaja['u_era5'],jaja['u_SD'],linewidth=0.4,linestyle='--',color='gray',zorder=1)
# sns.regplot(data=jaja, x='u_obs',y='their_res',ci=99)
# import matplotlib
# cmap=matplotlib.colormaps['prism']

# ax[0].text(0.05, 0.95, 'a.', transform=ax[0].transAxes,
#          fontsize=15, fontweight='bold', va='center', ha='center')
# ax[1].text(0.05, 0.95, 'b.', transform=ax[1].transAxes,
#          fontsize=15, fontweight='bold', va='center', ha='center')
# plt.axes(ax[0])
import seaborn as sns
# sns.kdeplot(data=jaja, x='u_obs',y='u_era5',ax=plt.gca(),color='dodgerblue',fill=True,levels=20,alpha=0.5,thresh=0.4,linewidth=0.5)
# sns.kdeplot(data=jaja, x='u_obs',y='u_SD',ax=plt.gca(),color='deeppink',fill=True,levels=20,alpha=0.5,thresh=0.3,linewidth=0.5)

# plt.plot(jaja['u_obs'],jaja['u_SD'],label="Model RMSE= "+str(round(np.mean(RMSE_ours),3))+r"$\text{ms}{}^{-1})$",linewidth=0.5,c='deeppink',zorder=2,alpha=0.9)
# plt.plot(jaja['u_obs'],jaja['u_era5'],label="ERA5L RMSE= "+str(round(np.mean(RMSE_era),3))+r"$\text{ms}{}^{-1})$",linewidth=0.5,c='dodgerblue',zorder=1,alpha=0.9)
# plt.scatter(jaja['u_obs'],jaja['u_SD'],label="Model",marker='^',edgecolor="black",s= 25,linewidth=0,c='deeppink',zorder=2,alpha=0.6)
# ax[0].errorbar(jaja['u_obs'],jaja['u_SD'],xerr=jaja['u_oerror'],yerr=jaja['u_error'],linewidth=0,elinewidth=0.1,color='deeppink',capsize=1,capthick=0.2,alpha=0.5)
# text1=str(round(np.mean(RMSE_ours),2))
# plt.scatter(jaja['u_obs'],jaja['u_era5'],label="ERA5L",edgecolor="black",s=25,linewidth=0,c='dodgerblue',zorder=1,alpha=0.6)
# text2=str(round(np.mean(RMSE_era),2))

# plt.annotate(text1,(1.8,5),ha='center',color='deeppink')
# plt.annotate(text2,(1.8,4.5),ha='center',color='dodgerblue')

# plt.axvline()
# plt.xlabel(r"$u^{obs}\:(\text{ms}{}^{-1})$")
# plt.ylabel(r"$u^{pred}\:(\text{ms}{}^{-1})$")
# plt.legend(loc='upper left',bbox_to_anchor=(0.0, 0.99),ncols=2)

# plt.xlim(-2,2)    
# plt.ylim(-2,2)
# jaja.dropna(inplace=True)

# ax[0].set_facecolor('white')  # Light gray background color
# # ax[0].yaxis.set_ticks_position('both')
# # ax[0].xaxis.set_ticks_position('both')
# # Show the plot
# ax[0].set_xlim(0,7)
# ax[0].set_ylim(0,7)
# # plt.minorticks_on()
# ax[0].set_xticks(np.arange(0,7,2))
# ax[0].set_xticks(np.arange(1,7,2),minor=True)
# ax[0].set_yticks(np.arange(2,7,2))
# ax[0].set_yticks(np.arange(1,7,2),minor=True)



df=pd.read_csv("RMSE_ours.csv",index_col=0)
RMSE_ours=df.values
df=pd.read_csv("RMSE_era5.csv",index_col=0)
RMSE_era=df.values
data = []

for rmse in RMSE_era:
    data.append({'Estimator': 'ERA5L', 'RMSE': rmse[0]})

for rmse in RMSE_ours:
    data.append({'Estimator': 'Model', 'RMSE': rmse[0]})

# Create DataFrame
boxplot_df = pd.DataFrame(data)

# plt.axes(ax1)

# sns.histplot(data=boxplot_df,hue='Estimator',x='RMSE',palette=palette,legend=False,alpha=0.5,fill='False',bins=10)
# ax1.tick_params(axis='y', which='major',pad=0)
# plt.yticks(rotation=90,size=8)
# plt.xticks(rotation=0,size=8)
# ax1.set_yticks(np.arange(0,30,10))
# ax1.set_yticks(np.arange(5,30,10),minor=True)
# ax1.set_xticks(np.arange(0,5,2))
# ax1.set_xticks(np.arange(1,5,2),minor=True)

# ax1.text(0.5, 0.95, "RMSE", transform=ax1.transAxes,
#         fontsize=8, fontweight='regular', va='top', ha='center')
# # ax1.text(0.5, 17, text1,
# #         fontsize=8, fontweight='regular', va='top', ha='center')
# # ax1.text(2.2, 10, text2,
# #         fontsize=8, fontweight='regular', va='top', ha='center')

# plt.ylabel("")
# plt.xlabel("")

# plt.savefig("Full_model_prediction.pdf")
# plt.show()



Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


C:\Users\krish\AppData\Local\Temp\ipykernel_16504\132001039.py:199: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  station_data['datetime'] = pd.to_datetime(station_data['datetime'])


Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


C:\Users\krish\AppData\Local\Temp\ipykernel_16504\132001039.py:135: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  x_validate_mean_wind['aspect_ratio_stddev_1'].loc[name]=0
C:\Users\krish\AppData\Local\Temp\ipykernel_16504\132001039.py:135: S

Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied
Ad-hoc threshold applied


In [47]:
print(np.mean(RMSE_ours))

0.6905082325448


In [48]:
print(betas)

               predictor      Beta        SE model_parameter      corr  \
0                  const  2.503347  0.510583   mean_wind_obs  0.510583   
1             relief_1km  0.004518  0.000802   mean_wind_obs -0.077739   
2             relief_5km -0.001567  0.000253   mean_wind_obs -0.421250   
3  aspect_ratio_stddev_1  0.112337  0.019797   mean_wind_obs -0.048599   
4                  const  0.133004  0.094004     sensitivity  0.094004   
5              station_z -0.000169  0.000054     sensitivity  0.110555   
6            Z_mean_10km  0.000217  0.000060     sensitivity  0.322800   
7                  const  1.364728  0.467699           delay  0.467699   
8         glacier_length -0.007639  0.031040           delay -0.018639   

     t_stat  
0  4.902917  
1  5.633185  
2  6.201579  
3  5.674334  
4  1.414871  
5  3.127260  
6  3.616306  
7  2.917964  
8  0.246106  


In [49]:
df = betas

# Mapping predictors to LaTeX variable names
latex_vars = {
    'const': '',
    'relief_1km': 'R_{1}',
    'relief_5km': 'R_{5}',
    'relief_10km': 'R_{10}',
    'aspect_ratio_stddev_1': '\\text{AR}_{\\sigma,1}',
    'aspect_ratio_relief_1': '\\text{AR}_{R,1}',
    'station_z': 'Z_s',
    'stddev_5km': '\\sigma_5',
    'stddev_1km': '\\sigma_1',
    'Z_mean_10km': '\\bar{Z}_{10}',
    'Slope_100m': 'S_{0.1}',
    'glacier_length':'L',
    'CW':'W_c',
    'mean_temp_era5':'\\bar{T}^{era}',
    'T_range_continentality':'K'
    
}

def sci_notation(val, err, var):
    # Determine appropriate scale
    max_val = max(abs(val), abs(err))
    if max_val == 0:
        return f"(0.00\\pm0.00) {var}"
    exp = int(np.floor(np.log10(max_val)))
    if -2 <= exp <= 1:
        # No scientific notation
        sign = '+' if val >= 0 else '-'
        return f" {sign} ({abs(val):.2f}\\pm{err:.2f}) {var}"
    else:
        scale = 10**exp
        val_scaled = val / scale
        err_scaled = err / scale
        sign = '+' if val >= 0 else '-'
        return f" {sign} ({abs(val_scaled):.2f}\\pm{err_scaled:.2f})\\times 10^{{{exp}}} {var}"

# Generate equation terms
def term(row):
    beta = row['Beta']
    se = row['SE']
    pred = row['predictor']
    if pred == 'const':
        return f"({beta:.2f}\\pm{se:.2f})"
    else:
        return sci_notation(beta, se, latex_vars[pred])

# Build the equations
equations = {}
for param in df['model_parameter'].unique():
    rows = df[df['model_parameter'] == param]
    terms = [term(row) for _, row in rows.iterrows()]
    lhs = {
        'mean_wind_obs': '\\bar{u}',
        'sensitivity': 's',
        'delay': '\\tau'
    }[param]
    equations[lhs] = f"{lhs} &= " + " ".join(terms)

# Output LaTeX
print("\\begin{align}")
for eq in equations.values():
    print(eq + " \\\\")
print("\\end{align}")

\begin{align}
\bar{u} &= (2.50\pm0.51)  + (4.52\pm0.80)\times 10^{-3} R_{1}  - (1.57\pm0.25)\times 10^{-3} R_{5}  + (0.11\pm0.02) \text{AR}_{\sigma,1} \\
s &= (0.13\pm0.09)  - (1.69\pm0.54)\times 10^{-4} Z_s  + (2.17\pm0.60)\times 10^{-4} \bar{Z}_{10} \\
\tau &= (1.36\pm0.47)  - (0.01\pm0.03) L \\
\end{align}


In [61]:
params.data_length_days.max()

586

\textbf{Name} & {$T_{{era}}$} & {$T_{{dew}}$} & {$e$} & {$e_s$} & {$P$} & {$\bar{u}^{{obs}}$} & {$Q^{{obs}}_{{shf}}$} & {$Q^{{obs}}_{{lhf}}$} & {$Q^{{obs}}_{{thf}}$} & {$\bar{u}^{{pred}}$} & {$Q^{{pred}}_{{shf}}$} & {$Q^{{pred}}_{{lhf}}$} & {$Q^{{pred}}_{{thf}}$} & {$\bar{u}^{{era}}$} & {$Q^{{era}}_{{shf}}$} & {$Q^{{era}}_{{lhf}}$} & {$Q^{{era}}_{{thf}}$} \\

# Where to stop

In [ ]:
set_plot_style()
# Data for Mean Wind Speed (extracted from image_b1b64b.png)
mean_wind_data = {
    'N_p': [1, 2, 3, 4, 5],
    'RMSE': [0.599815,0.544945, 0.389277,0.369719, 0.333732]
}

df_mean_wind = pd.DataFrame(mean_wind_data)

# Data for Diurnal Wind Speed (extracted from image_b1b687.png)
diurnal_wind_data = {
    'N_p': [2, 3, 4, 5],
    'RMSE': [0.582, 0.528, 0.512, 0.506]
}
df_diurnal_wind = pd.DataFrame(diurnal_wind_data)

# Apply plotting style
set_plot_style()

# Create a new subplot for RMSE plots
fig, ax = fig_row(cols=2) # Adjusted figsize for better readability
    
# --- Left Subplot (a.) - Mean Wind Speed RMSE ---
ax[0].plot(df_mean_wind['N_p'], df_mean_wind['RMSE'], linewidth=1.5, color='deeppink', marker='o')
ax[0].scatter(df_mean_wind['N_p'].iloc[2],df_mean_wind['RMSE'].iloc[2],s=100,color='red',alpha=0.5,zorder=-10)
ax[0].set_xlabel(r"$n$")
ax[0].set_ylabel(r"RMSE (ms$^{-1}$)")
ax[0].set_xticks(df_mean_wind['N_p']) # Ensure ticks are at N_p values
ax[0].text(0.05, 0.95, 'a.', transform=ax[0].transAxes,
           fontsize=15, fontweight='bold', va='center', ha='center')
ax[0].legend(frameon=False, shadow=True, fontsize=10, loc='upper right')
plt.subplots_adjust(wspace=0.3,left=0.1)


ax[0].set_ylim(0.25,0.65)
ax[0].set_yticks(ticks=np.arange(0.3,0.65,0.1))
ax[0].set_yticks(ticks=np.arange(0.25,0.65,0.1),minor=True)
plt.axes(ax[0])
plt.tick_params(axis='y',rotation=90)


# --- Right Subplot (b.) - Diurnal Wind Speed RMSE ---
ax[1].plot(df_diurnal_wind['N_p'], df_diurnal_wind['RMSE'], linewidth=1.5, color='deeppink', marker='o')
ax[1].scatter(df_diurnal_wind['N_p'].iloc[1],df_diurnal_wind['RMSE'].iloc[1],s=100,color='red',alpha=0.5,zorder=-10)
ax[1].set_xlabel(r"$n$")
ax[1].set_ylabel(r"RMSE (ms$^{-1}$)")
ax[1].set_xticks(df_diurnal_wind['N_p']) # Ensure ticks are at N_p values
ax[1].text(0.05, 0.95, 'b.', transform=ax[1].transAxes,
           fontsize=15, fontweight='bold', va='center', ha='center')
ax[1].legend(frameon=False, shadow=True, fontsize=10, loc='upper right')
ax[1].set_ylim(0.475,0.61)
ax[1].set_yticks(ticks=np.arange(0.5,0.61,0.1))
ax[1].set_yticks(ticks=np.arange(0.475,0.6,0.025),minor=True)
plt.axes(ax[1])
plt.tick_params(axis='y',rotation=90)
plt.subplots_adjust(wspace=0.2,left=0.1)
# plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()
plt.savefig("FigureS2.pdf",bbox_inches='tight')

C:\Users\krish\AppData\Local\Temp\ipykernel_18716\4088647037.py:31: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax[0].legend(frameon=False, shadow=True, fontsize=10, loc='upper right')
C:\Users\krish\AppData\Local\Temp\ipykernel_18716\4088647037.py:50: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax[1].legend(frameon=False, shadow=True, fontsize=10, loc='upper right')


In [ ]:
35.494211098072086
116.41194112144976
182.0521814765206
321.3407194013768
220.41629334317807
293.0591724769632
85.40097126737743
340.5854418218436
39.53133087538233
113.54878650331207
111.61810752106439
114.83606064974495
77.541417567609
186.0075735808753
187.9890132523121
349.9741280128226
0.30977119910756484
42.01990959437059
71.90754965442599
263.29308628160993
78.6976756178936
229.9515074352231
289.16342469481685
33.152810051551576
54.232219486083274
5.477044526201558
18.828819435902965
199.16435947671766

# Figure S1: defused

In [ ]:
# import Cryosphere_defused

In [ ]:
params

In [ ]:
for name in params.index:
    print(name, params['lat'].loc[name],",",params['lon'].loc[name])

In [ ]:
Turbulence_table.columns

In [ ]:
latex_table=Turbulence_table[['Tera','Tdewp','e','es','P','uobs','upred','uera5','EH(obs)','EH (param)','EH (ERA5L)','EL(obs)','EL (param)','EL (ERA5L)','QH(obs)','QH(param)','QH(ERA5L)']].reset_index()

In [ ]:
latex_table.to_latex("Thf_table.tex",float_format="%.1f")

In [ ]:
params['station_z']

In [13]:
%matplotlib qt
import pandas as pd
data=pd.read_csv("Universaldatasets_smallerdomain/Djankuat.csv")
data['datetime']=pd.to_datetime(data['datetime'])
data.set_index('datetime',inplace=True)
data=data[data['ws']>-0.1]
data=data[data['temp']>-20]
data[['ws','temp']].plot()

<Axes: xlabel='datetime'>

In [14]:
params.mean_wind_obs.sort_values()

glacier_name
Lirung             0.959412
Camp2              1.101833
Yala               1.106627
Satopanth          1.191696
Trakarding         1.604360
Weissee            1.867991
Pyramid            1.958641
Kalapathar         1.963522
Kennikot           1.972768
Morimoto           1.992634
Chottashigri_BC    2.407583
Weart2             2.462002
Chottashigri       2.619027
Place4             2.953415
Place1             2.966270
Southcol           3.006970
Place2             3.095601
Bishop             3.179746
Place3             3.200515
Langenferner       3.366824
Arolla             3.382661
Balcony            3.419413
Weart1             3.451064
Bellavista         3.578075
Hinterisferner     3.885727
Schiaparelli       3.915630
Drangdrung         4.203627
Djankuat           5.119577
Name: mean_wind_obs, dtype: float64

In [32]:
plt.scatter(params.mean_temp_obs,params.mean_wind_obs)